# pre-processing and cleaning

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
#from sklearn.metrics import confusion_matrix #bugged because confusion_matrix([1,1],[1,1]) = [[2]], not [[2,0],[0,0]]. Also confusion_matrix([0,0],[0,0]) = [[2]]
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import KFold
import random as rd
from sklearn.decomposition import PCA
from sklearn import preprocessing
import copy
from sklearn import tree as sktree
from sklearn.metrics import accuracy_score
from scipy.optimize import curve_fit
from scipy.stats import norm
import seaborn as sns
from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import r2_score I make my own
from scipy.stats import rankdata
import warnings
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import svm, datasets
from sklearn import linear_model
import random
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score
from matplotlib.lines import Line2D
import operator as op
from functools import reduce

import torch
from torch import nn
from torch.autograd import Variable
from torch.optim import LBFGS
from torch.utils.data import Dataset, DataLoader

def confusion_matrix(y_true,y_predicted):
  true_positives = np.sum([i*j for i,j in zip(y_true,y_predicted)])
  false_negatives = np.sum([int((i==1)&(j==0)) for i,j in zip(y_true,y_predicted)])
  true_negatives = np.sum([(1-i)*(1-j) for i,j in zip(y_true,y_predicted)])
  false_positives = np.sum([int((i==0)&(j==1)) for i,j in zip(y_true,y_predicted)])
  return [[true_positives,false_positives],[false_negatives,true_negatives]]

def sensitivity_score(y_true,y_predicted):
  assert(len(y_true)==len(y_predicted))
  for i in y_true:
    assert((i == 1) | (i == 0))
  for i in y_predicted:
    assert((i == 1) | (i == 0))
  true_positives = np.sum([i*j for i,j in zip(y_true,y_predicted)])
  false_negatives = np.sum([int((i==1)&(j==0)) for i,j in zip(y_true,y_predicted)])
  if (true_positives+false_negatives) == 0:
    return np.nan
  return true_positives/(true_positives+false_negatives)

def specificity_score(y_true,y_predicted):
  assert(len(y_true)==len(y_predicted))
  for i in y_true:
    assert((i == 1) | (i == 0))
  for i in y_predicted:
    assert((i == 1) | (i == 0))
  true_negatives = np.sum([(1-i)*(1-j) for i,j in zip(y_true,y_predicted)])
  false_positives = np.sum([int((i==0)&(j==1)) for i,j in zip(y_true,y_predicted)])
  if (true_negatives+false_positives) == 0:
    return np.nan
  return true_negatives/(true_negatives+false_positives)

def r2_variance(y_true,y_predicted):
  assert(len(y_true)==len(y_predicted))
  for i in y_true:
    assert((i == 1) | (i == 0))
  return [(i-j)**2 for i,j in zip(y_true,y_predicted)]

def r2_score(y_true,y_predicted):
  assert(len(y_true)==len(y_predicted))
  for i in y_true:
    assert((i == 1) | (i == 0))  
  a = -3.25
  return np.mean([np.log(1+np.exp(a*(1-2*np.abs(i-j)))) for i,j in zip(y_true,y_predicted)])

def abs_differences(y_true,y_predicted,index = None):
  assert(len(y_true)==len(y_predicted))
  for i in y_true:
    assert((i == 1) | (i == 0))
  return pd.Series([np.abs(i-j) for i,j in zip(y_true,y_predicted)],index=index)

plt.rcParams['figure.figsize'] = [16,8]

def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min(), x.max()
    y_min, y_max = y.min(), y.max()
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

def binomial(size,rate):
  error = np.sqrt(rate*(1-rate)/size)
  return error
  
def out_of_data_z_score(size,rate,new_rate):
  return (new_rate-rate)/binomial(size,rate)

def comb(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom  # or / in Python 2comb(10,2)

## Plots

In [ ]:
#deprecated
def tau_plots(model_X,model_y,features_premade = [],show_features = [],random_features = True,features_each = []): #premade is always included, show_features are which part of premade you want to see (usually it won't show premade), features_each includes each feature you want to be a part of premade but you're not sure which and random_features adds other random features
  did_appear = {}
  for tau in tqdm(np.arange(0.5,7.0,0.1)):
    for i in range(10):
      random_features_list = []
      if random_features:
        random_features_list = [feature for feature in model_X.columns.drop(features_premade+features_each) if random.random() > 0.5]
      abc = random_features_list+features_premade
      if len(features_each) != 0:
        for each_feature in features_each:
          temp = hard_function(tau = tau, X = model_X[np.append(abc,each_feature)], y=model_y, estimator = estimator_models)
          if each_feature in temp:
            if each_feature not in did_appear.keys():
              did_appear[each_feature] = [tau]
            else:
              did_appear[each_feature].append(tau)
          else:
            if each_feature not in did_appear.keys():
              did_appear[each_feature] = [None]
            else:
              did_appear[each_feature].append(None)          
      else:
        temp = hard_function(tau = tau, X = model_X[abc], y=model_y, estimator = estimator_models)
      for feature in show_features+random_features_list:
        if feature in temp:
          if feature not in did_appear.keys():
            did_appear[feature] = [tau]
          else:
            did_appear[feature].append(tau)
        else:
          if feature not in did_appear.keys():
            did_appear[feature] = [None]
          else:
            did_appear[feature].append(None)        
  for i in did_appear.keys():
    fig = plt.figure(i)
    plt.plot(np.arange(len(did_appear[i])),did_appear[i])
    plt.title(i)
    plt.xlim([0,len(did_appear[i])])
    plt.ylim([0,7])
    plt.grid()
  return did_appear

def confidence_plot(y,y_low=-0.2,y_high = 1.2, y_step = 0.07, edge = 0.5, flip = True,**models):
  warnings.filterwarnings('ignore')
  for j,model in models.items():
    temp = [np.average(y[(model > i) & (model < i + y_step)]) for i in np.arange(y_low,y_high,y_step)]
    if flip:
      for k in range(len(np.arange(y_low,y_high,y_step))):
        if round(y_low + (k+1/2)*y_step,1) <= edge:
          temp[k] = 1-temp[k]
    fig1 = plt.figure(figsize=(24,8))

    plt.errorbar([np.str(round(i,2)) + "-" + np.str(round(i+y_step,2)) for i in np.arange(y_low,y_high,y_step)],temp,yerr = [1/np.sqrt(model[(model > i) & (model < i + y_step)].count()) for i in np.arange(y_low,y_high,y_step)],label=i)
    plt.ylim([0,1])

  if flip:
    plt.title(f'Confidence in result versus y_predicted value, edge = {edge}')
  else:
    plt.title(f'Average of y versus y_predicted value, edge = {edge}')
  plt.legend()
  plt.grid()
  plt.show()
  warnings.resetwarnings()

def population_y_predicted(y_low=-0.2,y_high = 1.2, y_step = 0.07,**models):
  warnings.filterwarnings('ignore')
  for i,model in models.items():
    print(i)
    temp = [model[(model > i) & (model < i + y_step)].count() for i in np.arange(y_low,y_high,y_step)]
    plt.figure(figsize=(24,8))
    plt.bar([np.str(round(i,2)) + "-" + np.str(round(i+y_step,2)) for i in np.arange(y_low,y_high,y_step)],temp,label=i)
    plt.legend()
    plt.grid()
    plt.show()
  warnings.resetwarnings()

def delta_accuracy(model1,model2,step = 0.01,ymin=0.0,ymax=1.0):
  accuracy_deltas = []
  accuracy_deltas_err = []
  for i in np.arange(ymin,ymax + step,step):
    _,numbers = loading_scores(model1,y,pad=False,edge=i,b_compare=model2,show=False,estimator=estimator_models)
    accuracy_delta = numbers["Delta_Accuracy"]
    accuracy_delta_err = numbers["Delta_Accuracy_Error"]
    accuracy_deltas.append(accuracy_delta)
    accuracy_deltas_err.append(accuracy_delta_err)
  fig = plt.figure()
  x_range = [i for i in np.arange(ymin,ymax+step,step)]
  plt.errorbar(x_range,accuracy_deltas,yerr=accuracy_deltas_err)
  plt.title(f"Accuracy difference vs Edge, first_model_accuracy - second_model_accuracy") 
  plt.plot(x_range,0*np.ones_like(x_range))
  plt.grid()
  plt.show()
  plt.close()

def delta_sensitivity(model1,model2,step = 0.01,ymin=0.1,ymax=0.5):
  sensitivity_deltas = []
  sensitivity_deltas_err = []
  for i in np.arange(ymin,ymax + step,step):
    _,numbers = loading_scores(model1,y,pad=False,edge=i,b_compare=model2,show=False,estimator=estimator_models)
    sensitivity_delta = numbers['Delta_Sensitivity']
    sensitivity_delta_err = numbers['Delta_Sensitivity_Error']
    sensitivity_deltas.append(sensitivity_delta)
    sensitivity_deltas_err.append(sensitivity_delta_err)
  fig = plt.figure()
  x_range = [i for i in np.arange(ymin,ymax+step,step)]
  plt.errorbar(x_range,sensitivity_deltas,yerr=sensitivity_deltas_err)
  plt.title(f"Sensitivity difference vs Edge, first_model_sensitivity - second_model_sensitivity") 
  plt.plot(x_range,0*np.ones_like(x_range))
  plt.grid()
  plt.show()
  plt.close()
  return sensitivity_deltas,sensitivity_deltas_err

def delta_specificity(model1,model2,step = 0.01,ymin=0.1,ymax=0.5):
  specificity_deltas = []
  specificity_deltas_err = []
  for i in np.arange(ymin,ymax + step,step):
    _,numbers = loading_scores(model1,y,pad=False,edge=i,b_compare=model2,show=False,estimator=estimator_models)
    specificity_delta = numbers['Delta_Specificity']
    specificity_delta_err = numbers['Delta_Specificity_Error']
    specificity_deltas.append(specificity_delta)
    specificity_deltas_err.append(specificity_delta_err)
  fig = plt.figure()
  x_range = [i for i in np.arange(ymin,ymax+step,step)]
  plt.errorbar(x_range,specificity_deltas,yerr=specificity_deltas_err)
  plt.title(f"Specificity difference vs Edge, first_model_specificity - second_model_specificity") 
  plt.plot(x_range,0*np.ones_like(x_range))
  plt.grid()
  plt.show()
  plt.close()
  return specificity_deltas,specificity_deltas_err

def sensitivity_specificity_edge(model,step = 0.01,ymin=0.0,ymax=1.0,show=False,one_point_one_Sample=False):
  sensitivitys_specificitys = []
  if one_point_one_Sample:
    abc = sorted(model)
    _,numbers = loading_scores(model,y,pad=False,edge=abc[0]/2,show=False,estimator=estimator_models)
    sensitivity = numbers["Sensitivity"]
    specificity = numbers["Specificity"]
    sensitivitys_specificitys.append([sensitivity,specificity,abc[0]/2])
    for i in range(len(abc)-1):
      _,numbers = loading_scores(model,y,pad=False,edge=(abc[i]+abc[i+1])/2,show=False,estimator=estimator_models)
      sensitivity = numbers["Sensitivity"]
      specificity = numbers["Specificity"]
      sensitivitys_specificitys.append([sensitivity,specificity,abc[i]])
    _,numbers = loading_scores(model,y,pad=False,edge=(1.0+abc[-1])/2,show=False,estimator=estimator_models)
    sensitivity = numbers["Sensitivity"]
    specificity = numbers["Specificity"]
    sensitivitys_specificitys.append([sensitivity,specificity,(1.0+abc[-1])/2])
  else:
    for i in np.arange(ymin,ymax + step,step):
      _,numbers = loading_scores(model,y,pad=False,edge=i,show=False,estimator=estimator_models)
      sensitivity = numbers["Sensitivity"]
      specificity = numbers["Specificity"]
      sensitivitys_specificitys.append([sensitivity,specificity,i])
  return sensitivitys_specificitys

def sensitivity_accuracy(step = 0.0025,ymin=0.1,ymax=0.7,sensitivity_range=[0.75,0.95],accuracy_range=[0.70,0.90],xerr=False,yerr=False,**models):
  fig = plt.figure()
  all_sensitivitys = {}
  all_sensitivitys_err = {}
  all_accuracies = {}
  all_accuracies_err = {}
  all_edges = {}
  out = {}
  for i,model in models.items():
    edges = []
    sensitivitys = []
    sensitivitys_err = []
    accuracies = []
    accuracies_err = []
    for edge in np.arange(ymin,ymax + step,step):
      _,numbers = loading_scores(model,y,pad=False,edge=edge,show=False,estimator=estimator_models)
      accuracy = numbers["Accuracy"]
      accuracy_err = numbers["Accuracy_Error"]
      sensitivity = numbers["Sensitivity"]
      sensitivity_err = numbers["Sensitivity_Error"]
      accuracies.append(accuracy)
      accuracies_err.append(accuracy_err)
      sensitivitys.append(sensitivity)
      sensitivitys_err.append(sensitivity_err)
      edges.append(edge)
    all_sensitivitys[i] = sensitivitys
    all_sensitivitys_err[i] = sensitivitys_err
    all_accuracies[i] = accuracies
    all_accuracies_err[i] = accuracies_err
    all_edges[i] = edges
    out[i] = [[a,b,c] for a,b,c in zip(sensitivitys,accuracies,edges)]
    if xerr:
      if yerr:
        #plt.errorbar(sensitivitys,accuracies,label = i)
        plt.errorbar(sensitivitys,accuracies,xerr = sensitivitys_err,yerr=accuracies_err,label = i)
        #plt.fill_between(np.array(sensitivitys) - np.array(accuracies_err), np.array(accuracies)-np.array(accuracies_err), np.array(accuracies)+np.array(accuracies_err),color='tab:cyan')
        #plt.fill_between(np.array(sensitivitys) - 0.5*np.array(accuracies_err), np.array(accuracies)-np.array(accuracies_err), np.array(accuracies)+np.array(accuracies_err),color='tab:cyan')
        #plt.fill_between(np.array(sensitivitys), np.array(accuracies)-np.array(accuracies_err), np.array(accuracies)+np.array(accuracies_err),color='tab:cyan')
        #plt.fill_between(np.array(sensitivitys) + 0.5*np.array(accuracies_err), np.array(accuracies)-np.array(accuracies_err), np.array(accuracies)+np.array(accuracies_err),color='tab:cyan')
        #plt.fill_between(np.array(sensitivitys) + np.array(accuracies_err), np.array(accuracies)-np.array(accuracies_err), np.array(accuracies)+np.array(accuracies_err),color='tab:cyan')
      else:
        plt.errorbar(sensitivitys,accuracies,xerr = sensitivitys_err,label = i)
    else:
      if yerr:
        plt.errorbar(sensitivitys,accuracies,yerr=accuracies_err,label = i)
      else:
        plt.errorbar(sensitivitys,accuracies,label = i)
    #plt.plot(sensitivitys,accuracies,label = i)
  plt.title(f"Accuracy vs Sensitivity")
  plt.xlim(sensitivity_range)
  plt.ylim(accuracy_range)
  plt.xlabel("Sensitivity")
  plt.ylabel("Accuracy")
  plt.legend()
  plt.grid()
  plt.show()
  plt.close()
  return out

def sensitivity_delta_accuracy(model1,model2,step = 0.0025,ymin=0.0,ymax=1.0,sensitivity_range=[0.0,1.0],accuracy_delta_range=[-0.10,0.10],xerr=False,yerr=False):
  sensitivitys1 = sensitivity_specificity_edge(model1,step=step,ymin=ymin,ymax=ymax)
  sensitivitys2 = sensitivity_specificity_edge(model2,step=step,ymin=ymin,ymax=ymax)
  sensitivitys = []
  sensitivitys_err = []
  accuracy_deltas = []
  accuracy_deltas_err = []
  for i in np.arange(ymin,ymax + step,step):
    epsilon = 1.0
    for sensitivity1,_,edge1 in sensitivitys1:
      if np.abs(edge1 - i) < epsilon:
        closest_sensitivity1 = sensitivity1
        epsilon = np.abs(edge1 - i)
    epsilon = 1.0
    for sensitivity2,_,edge2 in sensitivitys2:
      if np.abs(sensitivity2 - closest_sensitivity1) < epsilon:
        closest_edge = edge2
        epsilon = np.abs(sensitivity2 - closest_sensitivity1)
    edge2 = closest_edge
    _,numbers = loading_scores(model1,y,pad=False,edge=i,b_compare=model2,show=False,edge2=edge2,estimator=estimator_models)
    sensitivity = numbers["Sensitivity"]
    sensitivity_err = numbers["Sensitivity_Error"]
    accuracy_delta = numbers["Delta_Accuracy"]
    accuracy_delta_err = numbers["Delta_Accuracy_Error"]
    accuracy_deltas.append(accuracy_delta)
    accuracy_deltas_err.append(accuracy_delta_err)
    sensitivitys.append(sensitivity)
    sensitivitys_err.append(sensitivity_err)
  fig = plt.figure()
  if xerr:
    if yerr:
      plt.errorbar(sensitivitys,accuracy_deltas,xerr = sensitivitys_err,yerr=accuracy_deltas_err)
    else:
      plt.errorbar(sensitivitys,accuracy_deltas,xerr = sensitivitys_err)
  else:
    if yerr:
      plt.errorbar(sensitivitys,accuracy_deltas,yerr=accuracy_deltas_err)
    else:
      plt.errorbar(sensitivitys,accuracy_deltas)
  plt.title(f"Accuracy difference vs Sensitivity, first_model_accuracy - second_model_accuracy") 
  plt.plot(sensitivitys,0*np.ones_like(sensitivitys))
  plt.xlim(sensitivity_range)
  plt.ylim(accuracy_delta_range)
  plt.grid()
  plt.show()
  plt.close()
  return sensitivitys,sensitivitys_err,accuracy_deltas,accuracy_deltas_err

#this one tries not to have 'correlations'. Delta here means edge - edge', delta means model1 - model2.
def Delta_sensitivity_delta_specificity(model1,model2,step = 0.0025,ymin=0.0,ymax=1.0,sensitivity_range=[0.0,1.0],specificity_delta_range=None,xerr=False,yerr=False):
  specificity_delta = 0.0
  sensitivitys1 = sensitivity_specificity_edge(model1,step=step,ymin=ymin,ymax=ymax)
  sensitivitys2 = sensitivity_specificity_edge(model2,step=step,ymin=ymin,ymax=ymax)
  sensitivitys = []
  sensitivitys_err = []
  specificity_deltas = []
  specificity_deltas_err = []
  for i in np.arange(ymin,ymax + step,step):
    epsilon = 1.0
    for sensitivity1,_,edge1 in sensitivitys1:
      if np.abs(edge1 - i) < epsilon:
        closest_sensitivity1 = sensitivity1
        epsilon = np.abs(edge1 - i)
    epsilon = 1.0
    for sensitivity2,_,edge2 in sensitivitys2:
      if np.abs(sensitivity2 - closest_sensitivity1) < epsilon:
        closest_edge = edge2
        epsilon = np.abs(sensitivity2 - closest_sensitivity1)
    edge2 = closest_edge
    _,numbers = loading_scores(model1,y,pad=False,edge=i,b_compare=model2,show=False,edge2=edge2,estimator=estimator_models)
    temp1 = specificity_delta
    sensitivity = numbers["Sensitivity"]
    sensitivity_err = numbers["Sensitivity_Error"]
    specificity_delta = numbers["Delta_Specificity"]
    specificity_delta_err = numbers["Delta_Specificity_Error"]
    specificity_deltas.append(specificity_delta-temp1)
    #accuracy_deltas_err.append(accuracy_delta_err)
    sensitivitys.append(sensitivity)
    #sensitivitys_err.append(sensitivity_err)
  fig = plt.figure()
  if xerr:
    if yerr:
      plt.errorbar(sensitivitys,specificity_deltas,xerr = sensitivitys_err,yerr=accuracy_deltas_err)
    else:
      plt.errorbar(sensitivitys,specificity_deltas,xerr = sensitivitys_err)
  else:
    if yerr:
      plt.errorbar(sensitivitys,specificity_deltas,yerr=accuracy_deltas_err)
    else:
      plt.errorbar(sensitivitys,specificity_deltas)
  plt.title(f"Delta Specificity difference vs Sensitivity, first_model - second_model") 
  plt.plot(sensitivitys,0*np.ones_like(sensitivitys))
  plt.xlim(sensitivity_range)
  if specificity_delta_range:
    plt.ylim(specificity_delta_range)
  plt.grid()
  plt.show()
  plt.close()
  return sensitivitys,sensitivitys_err,specificity_deltas,specificity_deltas_err

def delta_r2(model1,model2):
  _,numbers = loading_scores(model1,y,pad=False,b_compare=model2,show=False,estimator=estimator_models)
  r2_delta = numbers[8]
  r2_delta_err = numbers[9]
  return f"R^2 of model 1 - R^2 of model 2 = {r2_delta}+-{r2_delta_err}"

def ROC(step = 0.0025,ymin=0.0,ymax=1.0,specificity_range=None,sensitivity_range=None,**models):
  fig = plt.figure(figsize=[8,8])
  all_sensitivitys = {}
  all_sensitivitys_err = {}
  all_specificitys = {}
  all_specificitys_err = {}
  all_edges = {}
  for i,model in models.items():
    edges = []
    sensitivitys = []
    sensitivitys_err = []
    specificitys = []
    specificitys_err = []
    for edge in np.arange(ymin,ymax + step,step):
      _,numbers = loading_scores(model,y,pad=False,edge=edge,show=False,estimator=estimator_models)
      specificity = numbers["Specificity"]
      specificity_err = numbers["Specificity_Error"]
      sensitivity = numbers["Sensitivity"]
      sensitivity_err = numbers["Sensitivity_Error"]
      specificitys.append(specificity)
      specificitys_err.append(specificity_err)
      sensitivitys.append(sensitivity)
      sensitivitys_err.append(sensitivity_err)
      edges.append(edge)
    all_sensitivitys[i] = sensitivitys
    all_sensitivitys_err[i] = sensitivitys_err
    all_specificitys[i] = specificitys
    all_specificitys_err[i] = specificitys_err
    all_edges[i] = edges
    plt.plot(sensitivitys,specificitys,label = i)
  plt.title(f"Specificity vs Sensitivity")
  if sensitivity_range:
    plt.xlim(sensitivity_range)
  if specificity_range:
    plt.ylim(specificity_range)
  plt.xlabel("Sensitivity")
  plt.ylabel("Specificitys")
  plt.legend()
  plt.grid()
  plt.show()
  plt.close()
  return all_sensitivitys,all_specificitys,all_edges

def radial_delta_accuracy(model1,model2, ymin=0.0,ymax=1.0, step = 0.0025, smooth = 0, yerr=True,sensitivitys_specificitys1=None,sensitivitys_specificitys2=None,adjust_p_values = False,sensitivity_x = True):
  print('When deciding which model is better, we give Sensitivity and Specificity equal "weight".')
  if sensitivitys_specificitys1 is None:
    sensitivitys_specificitys1 = sensitivity_specificity_edge(model1,step=step,ymin=ymin,ymax=ymax,one_point_one_Sample=True)
  if sensitivitys_specificitys2 is None:
    sensitivitys_specificitys2 = sensitivity_specificity_edge(model2,step=step,ymin=ymin,ymax=ymax,one_point_one_Sample=True)

  r_temp = (0,0)
  delta_rs = []
  edges1 = []
  edges2 = []
  rs = []
  rs_err = []
  angles = []
  xx = []
  for sensitivity1,specificity1,edge1 in sensitivitys_specificitys1:
    closest_sensitivity1 = sensitivity1
    closest_specificity1 = specificity1
    found_edge1 = edge1
    #not needed because of one_point_one_Sample
    # #make sure we don't have the same point twice
    # if len(xx) > 0:
    #   if (np.round(closest_sensitivity1,5) != np.round(temp2,5)) | (np.round(closest_specificity1,5) != np.round(temp,5)):
    #     if sensitivity_x:
    #       xx.append(closest_sensitivity1)
    #     else:
    #       xx.append(found_edge1)
    #   else:
    #     continue
    # else:
    if sensitivity_x:
      xx.append(closest_sensitivity1)
    else:
      xx.append(found_edge1)
    temp = closest_specificity1
    temp2 = closest_sensitivity1
    edges1.append(found_edge1)
    epsilon = 1.0
    for sensitivity2,specificity2,edge2 in sensitivitys_specificitys2:
      if np.sqrt((sensitivity2 - closest_sensitivity1)**2 + (specificity2 - closest_specificity1)**2) < epsilon: #if the sensitivity and specificity have 'equal error bars', then we just find the closest one
        closest_edge = edge2
        epsilon = np.sqrt((sensitivity2 - closest_sensitivity1)**2 + (specificity2 - closest_specificity1)**2)
    edge2 = closest_edge
    edges2.append(edge2)
    _,numbers = loading_scores(model1,y,pad=False,edge=edge1,b_compare=model2,show=False,edge2=edge2,estimator=estimator_models)
    delta_sensitivity = -numbers["Delta_Sensitivity"] # don't ask me why this is -
    delta_sensitivity_err = numbers["Delta_Sensitivity_Error"]
    delta_specificity = numbers["Delta_Specificity"]
    delta_specificity_err = numbers["Delta_Specificity_Error"]
    r = np.sqrt(delta_sensitivity**2 + delta_specificity**2)
    delta_r = (delta_sensitivity - r_temp[0],delta_specificity - r_temp[1])
    delta_r = np.sqrt(np.abs(np.sign(delta_r[0])*delta_r[0]**2 + np.sign(delta_r[1])*delta_r[1]**2)) # here when 'positive' Sensitivity becomes 'positive' Specificity, we don't count it.
    delta_rs.append(delta_r)
    r_temp = (delta_sensitivity,delta_specificity)
    if r != 0.0:
      df_1 = delta_sensitivity/r
      df_2 = delta_specificity/r
      r_err = np.sqrt(df_1**2*delta_sensitivity_err**2 + df_2**2*delta_specificity_err**2)
    else:
      r_err = np.sqrt(1/2*delta_sensitivity_err**2 + 1/2*delta_specificity_err**2) # here it depends on the direction that r=0 is achieved. We use 45 degrees.
    rs.append(r)
    rs_err.append(r_err)
    if (delta_specificity == 0.0) & (delta_sensitivity > 0.0):
      angle = 0.0
    elif (delta_specificity == 0.0) & (delta_sensitivity < 0.0):
      angle = 180.0
    else: 
      if r != 0.0:
        angle = 180.0/np.pi*2.0*np.arctan(delta_specificity/(delta_sensitivity+r)) #this goes from -180 to 180
      else:
        angle = np.nan
    angles.append(angle)

  if smooth > 0:
    print(f"'smooth' {2*smooth + 1} points: We think about angles in two opposite buckets: model 1 better and model 2 better. Really these have to be added as Vectors.")
    print("'smooth': Use only for points with the same direction.")
    rs_err = [i**2 for i in rs_err]
    # i_max = len(rs) - 2*two_smooth - 1
    # j_max = 2*two_smooth - 1
    # i_max + j_max + 1 = len(rs) - 1
    for i in range(len(rs)-2*smooth):
      for j in range(2*smooth):
        rs[i] += rs[i+j+1]
        rs_err[i] += rs_err[i+j+1] # these need to be added like x**2
        xx[i] += xx[i+j+1]
    rs = rs[:-2*smooth]
    rs_err = rs_err[:-2*smooth]
    xx = xx[:-2*smooth]
    rs = [i/(2*smooth + 1) for i in rs]
    rs_err = [np.sqrt(i/(2*smooth + 1)) for i in rs_err]
    xx = [i/(2*smooth + 1) for i in xx]
    edges1 = edges1[smooth:-smooth]
    edges2 = edges2[smooth:-smooth]
    angles = angles[smooth:-smooth]

  col = []
  for i in range(len(xx)):
    if (-180<=angles[i]<-45) or (135<angles[i]<=180): # we use Sensitivity == Specificity in our cost.
        col.append('blue')  
    elif -45<angles[i]<=135:
        col.append('orange') 
        rs[i] = -rs[i]
        delta_rs[i] = -delta_rs[i]
    else:
        col.append('green')  # this is np.nan for r=0 
  
  fig2 = plt.figure()
  plt.scatter(xx,angles,c=col)
  plt.plot(xx,angles)
  plt.title(f"Direction from first model to closest point on second model") 
  if sensitivity_x:
    plt.xlabel("Sensitivity1")
  else:
    plt.xlabel("Edge1")
  plt.ylabel("Angle(-180 to 180 degrees)")  
  plt.grid()
  legend_elements = [Line2D([0], [0], marker='o', color='w', label='first_better',
                          markerfacecolor='blue', markersize=8),
                     Line2D([0], [0], marker='o', color='w', label='second_better',
                          markerfacecolor='orange', markersize=8),
                     Line2D([0], [0], marker='o', color='w', label='unsure',
                          markerfacecolor='green', markersize=8)]
  plt.legend(handles=legend_elements, loc="upper left")  
  plt.show()

  fig = plt.figure()
  plt.scatter(xx,rs,c=col)
  if yerr:
    plt.errorbar(xx,rs,yerr=rs_err)
  else:
    plt.errorbar(xx,rs)
  if sensitivity_x:
    plt.xlabel("Sensitivity1")
    plt.title(f"Radial Delta Accuracy vs Sensitivity1, first_model - second_model") 
  else:
    plt.xlabel("Edge1")
    plt.title(f"Radial Delta Accuracy vs Edge1, first_model - second_model") 
  plt.ylabel("Radial Delta Accuracy")
  plt.grid()
  legend_elements = [Line2D([0], [0], marker='o', color='w', label='first_better',
                          markerfacecolor='blue', markersize=8),
                     Line2D([0], [0], marker='o', color='w', label='second_better',
                          markerfacecolor='orange', markersize=8),
                     Line2D([0], [0], marker='o', color='w', label='unsure',
                          markerfacecolor='green', markersize=8)]
  plt.legend(handles=legend_elements, loc="upper left")  
  plt.show()

  for i in range(len(rs_err)):
    if (np.isnan(rs_err[i])) & (rs[i]==0.0):
      rs_err[i] = 0.000001
  z_scores = [i/j for i,j in zip(rs,rs_err)]
  if adjust_p_values:
    z_scores = adjusted_p_values(z_scores)
  scatter = plt.scatter(xx,z_scores,c=col)
  plt.plot(xx,z_scores)
  legend_elements = [Line2D([0], [0], marker='o', color='w', label='first_better',
                          markerfacecolor='blue', markersize=8),
                     Line2D([0], [0], marker='o', color='w', label='second_better',
                          markerfacecolor='orange', markersize=8),
                     Line2D([0], [0], marker='o', color='w', label='unsure',
                          markerfacecolor='green', markersize=8)]
  plt.legend(handles=legend_elements, loc="upper left")
  plt.title(f"Z-scores") 
  if sensitivity_x:
    plt.xlabel("Sensitivity1")
  else:
    plt.xlabel("Edge1")
  plt.ylabel("Z-scores")
  plt.grid()
  plt.show()

  fig = plt.figure()
  # if sensitivity_x:
  #   if yerr:
  #     plt.scatter(xx,delta_rs,c=col)
  #     #plt.errorbar(xx,rs,yerr=rs_err)
  #   else:
  #     plt.scatter(xx,delta_rs,c=col)
  #     plt.errorbar(xx,delta_rs)
  #   plt.xlabel("Sensitivity1")
  #   plt.title(f"Delta Radial Delta Accuracy vs Sensitivity1, first_model - second_model") 
  # else:
  plt.scatter(range(len(delta_rs)),delta_rs,c=col)
  if sensitivity_x:
    plt.xlabel("Sensitivity1")
  else:
    plt.xlabel("Edge1")
  plt.xticks(np.arange(0, len(delta_rs), step=20), np.round(xx[::20],3))
  plt.title(f"Delta Radial Delta Accuracy, each point is a person, first_model - second_model") 
  plt.ylabel("Delta Radial Delta Accuracy")
  plt.grid()
  legend_elements = [Line2D([0], [0], marker='o', color='w', label='first_better',
                          markerfacecolor='blue', markersize=8),
                    Line2D([0], [0], marker='o', color='w', label='second_better',
                          markerfacecolor='orange', markersize=8),
                    Line2D([0], [0], marker='o', color='w', label='unsure',
                          markerfacecolor='green', markersize=8)]
  plt.legend(handles=legend_elements, loc="upper left")  
  plt.show()

  plt.close()

  result = {}
  result['delta_rs'] = delta_rs
  result['edges1'] = edges1
  result['edges2'] = edges2
  result['rs'] = rs
  result['rs_err'] = rs_err
  result['angles'] = angles
  result['sensitivitys_specificitys1'] = sensitivitys_specificitys1
  result['sensitivitys_specificitys2'] = sensitivitys_specificitys2

  return result

def fdr(p_vals):
  ranked_p_values = rankdata(p_vals)
  fdr = p_vals * len(p_vals) / ranked_p_values
  for i in range(len(fdr)-1):
    if fdr[i]>fdr[i+1]:
      fdr[i]=fdr[i+1]
  fdr[fdr > 1] = 1
  return fdr

def adjusted_p_values(z_scores):
  p_values = norm.sf(z_scores)*2 #twoside
  adjusted_p_values = fdr(p_values)
  adjusted_z_scores = norm.ppf(1-adjusted_p_values/2)
  return adjusted_z_scores

def log_likelihood(first_model,y,y_low=0.0,y_high = 1.0, y_step = 0.04, flip = True):
  warnings.filterwarnings('ignore')
  
  delta_y = np.abs(y - first_model)

  temp = [len([delta for delta in delta_y if ((delta > i) & (delta < i + y_step))]) for i in np.arange(y_low,y_high,y_step)]

  temp = [i/len(y) for i in temp]

  temp = np.log(temp)

  fig1 = plt.figure(figsize=(24,8))

  plt.errorbar([np.str(round(i,2)) + "-" + np.str(round(i+y_step,2)) for i in np.arange(y_low,y_high,y_step)],temp)
  
  regr = linear_model.LinearRegression()

  regr.fit(np.reshape([ i+y_step/2 for i in np.arange(y_low,y_high,y_step)],(-1, 1)), temp)

  line = regr.predict(np.reshape([ i+y_step/2 for i in np.arange(y_low,y_high,y_step)],(-1, 1)))

  plt.plot([np.str(round(i,2)) + "-" + np.str(round(i+y_step,2)) for i in np.arange(y_low,y_high,y_step)],line)

  plt.show()
  warnings.resetwarnings()

  return regr.coef_[0],regr.intercept_
  

## Estimators and Cross-correlation

In [ ]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

def log_odds_ratio(y):
  return np.log(y/(1-y))

def estimator_models(X,y,index,edge=0.5,b_compare=None,edge2=None):
  if edge2 is None:
    edge2 = edge
  b_predicted_2 = np.rint(X>edge)

  accuracy = accuracy_score(y,b_predicted_2)
  accuracy_err = np.sqrt(accuracy*(1-accuracy)/len(y)) #thinking of this as a binomial problem

  abs_differences_ = abs_differences(y,X,index)
  def likelihood_error(error,a):
    return 1/(1+np.exp(a*(1-2*error)))
  a = 3.3
  abs_differences_sum = np.sum([likelihood_error(i,a) for i in abs_differences_])
  abs_differences_err_sum = np.std([likelihood_error(i,a) for i in abs_differences_])*np.sqrt(len(abs_differences_))
  # this is really not the same likelihood function that you can use for y_abs_difference, but usually this is what people do
  y_abs_differences = [likelihood_error(np.abs(i-np.mean(y)),a) for i in y]
  y_abs_differences_sum = np.sum(y_abs_differences)
  y_abs_differences_err_sum = np.std(y_abs_differences)*np.sqrt(len(y_abs_differences))
  r2 = 1-abs_differences_sum/y_abs_differences_sum
  r2_err = np.sqrt(1.0/y_abs_differences_sum**2*abs_differences_err_sum**2+(abs_differences_sum/y_abs_differences_sum**2)**2*y_abs_differences_err_sum**2)

  sensitivity = sensitivity_score(y,b_predicted_2)
  sensitivity_sample = np.count_nonzero(y == 1)
  sensitivity_err = np.sqrt(sensitivity*(1-sensitivity)/sensitivity_sample)

  specificity = specificity_score(y,b_predicted_2)
  specificity_sample = np.count_nonzero(y == 0)
  specificity_err = np.sqrt(specificity*(1-specificity)/specificity_sample)

  confusion = np.array(confusion_matrix(y,b_predicted_2))

  auc = roc_auc_score(y,X)

  result = {}
  result['accuracy'] = accuracy
  result['accuracy_err'] = accuracy_err
  result['r2'] = r2
  result['r2_err'] = r2_err
  result['sensitivity'] = sensitivity
  result['sensitivity_err'] = sensitivity_err
  result['specificity'] = specificity
  result['specificity_err'] = specificity
  result['confusion'] = confusion
  result['auc'] = auc

  if b_compare is not None:
    b_compare_test = b_compare
    b_compare_test_2 = np.rint(b_compare > edge2)
    compare_accuracy = accuracy_score(y,b_compare_test_2)
    compare_r2 = r2_score(y,b_compare_test)
    sensitivity_0 = np.sum([np.rint((a==b) & (c == 1)) for a,b,c in zip(b_predicted_2,b_compare_test_2,y)])
    sensitivity_neg_1 = np.sum([np.rint((a<b) & (c == 1)) for a,b,c in zip(b_predicted_2,b_compare_test_2,y)])
    sensitivity_pos_1 = np.sum([np.rint((a>b) & (c == 1)) for a,b,c in zip(b_predicted_2,b_compare_test_2,y)])
    specificity_0 = np.sum([np.rint((a==b) & (c == 0)) for a,b,c in zip(b_predicted_2,b_compare_test_2,y)])
    specificity_neg_1 = np.sum([np.rint((a<b) & (c == 0)) for a,b,c in zip(b_predicted_2,b_compare_test_2,y)])
    specificity_pos_1 = np.sum([np.rint((a>b) & (c == 0)) for a,b,c in zip(b_predicted_2,b_compare_test_2,y)])
    delta_r2 = r2 - compare_r2
#    delta_r2_err = np.std() who needs this
    delta_accuracy = accuracy - compare_accuracy
    #this doesn't work because it's a - b which is not binomial
    #delta_accuracy_err = np.sqrt(accuracy*(1-accuracy)/len(y))
    delta_accuracy_err = np.std(b_predicted_2-b_compare_test_2)/np.sqrt(len(y))
    result['delta_accuracy'] = delta_accuracy
    result['delta_accuracy_err'] = delta_accuracy_err
    result['delta_r^2'] = delta_r2
#    result['delta_r^2_err'] = delta_r2_err
    result['delta_sensitivitys'] = [sensitivity_0,sensitivity_neg_1,sensitivity_pos_1]
    result['delta_specificitys'] = [specificity_0,specificity_neg_1,specificity_pos_1] 
    result['specificity_sample'] = specificity_sample
    result['sensitivity_sample'] = sensitivity_sample
  return result

def estimator_SVD(A_train,A_test,b_train,b_test,test_index,train_index,r = 0,q=0,p=0,edge=0.5,l2_reg=1.0): #q,p are not used
  if len(A_train.shape) == 1:
    b_predicted_test = A_test
    b_predicted_test_2 = [int(row > edge) for row in b_predicted_test]
    b_predicted_train = A_train
    b_predicted_train_2 = [int(row > edge) for row in b_predicted_train]
    x_fit = 1
    S = 1
  else:
    U,S,VT = np.linalg.svd(A_train,full_matrices=False)
    if r != 0:
      U = U[:,:r+1]
      S = S[:r+1]
      VT = VT[:r+1,:]
    S = np.diag(S)
    A_train_inv = VT.T @ np.linalg.inv(S) @ U.T
    x_fit = A_train_inv @ b_train
    b_predicted_test = A_test @ x_fit
    b_predicted_test_2 = [int(row > edge) for row in b_predicted_test]
    b_predicted_train = A_train @ x_fit
    b_predicted_train_2 = [int(row > edge) for row in b_predicted_train]
  test_accuracy = accuracy_score(b_test,b_predicted_test_2)
  train_accuracy = accuracy_score(b_train,b_predicted_train_2)
  #test_r2 = r2_score(b_test,b_predicted_test)
  test_abs_differences = abs_differences(b_test,b_predicted_test,test_index)
  train_abs_differences = abs_differences(b_train,b_predicted_train,train_index)
  test_sensitivity = sensitivity_score(b_test,b_predicted_test_2)
  train_sensitivity = sensitivity_score(b_train,b_predicted_train_2)
  test_specificity = specificity_score(b_test,b_predicted_test_2)
  train_specificity = specificity_score(b_train,b_predicted_train_2) 
  if np.isnan(test_sensitivity):
    test_sensitivity = 0 # this is if we have 0 'sample size'
  if np.isnan(train_sensitivity):
    train_sensitivity = 0
  if np.isnan(test_specificity):
    test_specificity = 0 # this is if we have 0 'sample size'
  if np.isnan(train_specificity):
    train_specificity = 0
  if np.isnan(test_accuracy):
    test_accuracy = 0 # this is if we have 0 'sample size'
  confusion = confusion_matrix(b_test,b_predicted_test_2)
  result = {}
  result['test_accuracy'] = test_accuracy
  result['train_accuracy'] = train_accuracy
  result['x_fit'] = x_fit
  #result['test_r^2'] = test_r2
  result['test_abs_differences'] = test_abs_differences
  result['train_abs_differences'] = train_abs_differences
  result['test_sensitivity'] = test_sensitivity
  result['train_sensitivity'] = train_sensitivity
  result['b_predicted_test_2'] = b_predicted_test_2
  result['S'] = S
  result['confusion'] = confusion
  result['test_specificity'] = test_specificity
  result['train_specificity'] = train_specificity
  result['b_predicted_test'] = b_predicted_test
  result['b_predicted_train'] = b_predicted_train
  return result

def cross_correlate_with_two_features(X,y,features1,features2,pad,estimator,method,nr_folds = 30, edge=0.5,c=None,gamma=None,show=False,svd_fit = True,svm_dim=2):
  result = {}
  result['test_accuracy'] = []
  result['train_accuracy'] = []
  result['x_fit'] = []
  result['test_abs_differences'] = pd.Series(dtype='float64',index=X.index)
  result['train_abs_differences'] = pd.DataFrame(dtype='float64',index=X.index)
  result['test_sensitivity'] = []
  result['train_sensitivity'] = []
  result['test_specificity'] = []
  result['train_specificity'] = []
  result['confusion'] = []
  result['test_sensitivity_sample'] = []
  result['test_specificity_sample'] = []
  result['train_sensitivity_sample'] = []
  result['train_specificity_sample'] = []
  result['b_train'] = []
  result['b_predicted_test'] = pd.Series(dtype='float64',index=X.index)
  result['which_model'] = []
  kf = KFold(n_splits=nr_folds,shuffle=True)
  gen = kf.split(X)
  gens = []
  for generator in gen:
    gens.append(generator)
  i = 0
  for generator in gens:
    i+=1
    A_train_1, A_test_1 = X[features1].iloc[generator[0]].to_numpy(), X[features1].iloc[generator[1]].to_numpy()
    A_train_2, A_test_2 = X[features2].iloc[generator[0]].to_numpy(), X[features2].iloc[generator[1]].to_numpy()
    test_index = X.iloc[generator[1]].index
    train_index = X.iloc[generator[0]].index
    b_train, b_test = np.rint(y.iloc[generator[0]].to_numpy()), np.rint(y.iloc[generator[1]].to_numpy())
    for i in b_train:
      assert((i == 1) | (i == 0))
    for i in b_test:
      assert((i == 1) | (i == 0))
    if pad:
      A_train_1 = np.pad(A_train_1,[(0,0),(0,1)],constant_values=1)
      A_test_1 = np.pad(A_test_1,[(0,0),(0,1)],constant_values=1)
      A_train_2 = np.pad(A_train_2,[(0,0),(0,1)],constant_values=1)
      A_test_2 = np.pad(A_test_2,[(0,0),(0,1)],constant_values=1)    
    out_1 = estimator(A_train_1,A_test_1,b_train,b_test,test_index,train_index,edge=edge)
    out_2 = estimator(A_train_2,A_test_2,b_train,b_test,test_index,train_index,edge=edge)
    b_train = pd.Series(b_train,index=train_index)
    b_test = pd.Series(b_test,index=test_index)
    b_predicted_test_1 = pd.Series(out_1['b_predicted_test'],index=test_index)
    b_predicted_train_1 = pd.Series(out_1['b_predicted_train'],index=train_index)
    b_predicted_test_2 = pd.Series(out_2['b_predicted_test'],index=test_index)
    b_predicted_train_2 = pd.Series(out_2['b_predicted_train'],index=train_index)

    if method == 'svm':
      if svm_dim == 2:
        X_k_neighbors_train = np.column_stack((b_predicted_train_1,b_predicted_train_2)) # using both models for svm
        X_k_neighbors_test = np.column_stack((b_predicted_test_1,b_predicted_test_2))
      else:
        X_k_neighbors_train = b_predicted_train_1.to_numpy().reshape(-1,1) # using just model1 for svm
        X_k_neighbors_test = b_predicted_test_1.to_numpy().reshape(-1,1)
      b_best_train = pd.Series(dtype='float64',index=train_index)
      for i in train_index:
        if np.abs(b_predicted_train_1[i] - b_train[i]) < np.abs(b_predicted_train_2[i] - b_train[i]):
          b_best_train.loc[i] = 1.0
        else:
          b_best_train.loc[i] = 0.0
      if gamma is None:
        gamma='scale'
      if c is None:
        c = 1.0
      clf = svm.SVC(kernel='rbf', gamma=gamma, C=c)
      #clf = svm.SVC(kernel='poly', gamma=gamma, C=c,degree=2) # this is really slow
      clf.fit(X_k_neighbors_train, b_best_train)
      which_model_train = pd.Series(clf.predict(X_k_neighbors_train),index=train_index)
      which_model_test = pd.Series(clf.predict(X_k_neighbors_test),index=test_index)
      
      def helper(a,b,c):
        assert ((a==1.0) | (a == 0.0))
        if a == 1.0:
          return b
        else:
          return c

      def divide(which_model,index):
        group1 = []
        group2 = []
        for i,j in zip(which_model,index):
          if i == 1.0:
            group1.append(j)
          else:
            group2.append(j)
        return group1,group2

      if svd_fit:    
        group1_train,group2_train = divide(which_model_train,train_index)
        group1_test,group2_test = divide(which_model_test,test_index)

        b_predicted_train_1_group_1 = b_predicted_train_1.loc[group1_train]
        b_predicted_train_2_group_1 = b_predicted_train_2.loc[group1_train]
        b_predicted_train_group_1 = np.column_stack((b_predicted_train_1_group_1.to_numpy(),b_predicted_train_2_group_1.to_numpy()))
        b_predicted_train_1_group_2 = b_predicted_train_1.loc[group2_train]
        b_predicted_train_2_group_2 = b_predicted_train_2.loc[group2_train]
        b_predicted_train_group_2 = np.column_stack((b_predicted_train_1_group_2.to_numpy(),b_predicted_train_2_group_2.to_numpy()))
        b_predicted_test_1_group_1 = b_predicted_test_1.loc[group1_test]
        b_predicted_test_2_group_1 = b_predicted_test_2.loc[group1_test]
        b_predicted_test_group_1 = np.column_stack((b_predicted_test_1_group_1.to_numpy(),b_predicted_test_2_group_1.to_numpy()))
        b_predicted_test_1_group_2 = b_predicted_test_1.loc[group2_test]
        b_predicted_test_2_group_2 = b_predicted_test_2.loc[group2_test]
        b_predicted_test_group_2 = np.column_stack((b_predicted_test_1_group_2.to_numpy(),b_predicted_test_2_group_2.to_numpy()))
        out_group_1 = estimator_SVD(b_predicted_train_group_1,b_predicted_test_group_1,b_train.loc[group1_train],b_test.loc[group1_test],group1_test,group1_train,edge=edge)
        out_group_2 = estimator_SVD(b_predicted_train_group_2,b_predicted_test_group_2,b_train.loc[group2_train],b_test.loc[group2_test],group2_test,group2_train,edge=edge)
        b_predicted_test_group_1 = pd.Series(out_group_1['b_predicted_test'],index=group1_test)
        b_predicted_train_group_1 = pd.Series(out_group_1['b_predicted_train'],index=group1_train)
        b_predicted_test_group_2 = pd.Series(out_group_2['b_predicted_test'],index=group2_test)
        b_predicted_train_group_2 = pd.Series(out_group_2['b_predicted_train'],index=group2_train)
        b_predict_train = pd.Series(index=train_index)
        b_predict_test = pd.Series(index=test_index)
        b_predict_train.loc[b_predicted_train_group_1.index] = b_predicted_train_group_1.values
        b_predict_train.loc[b_predicted_train_group_2.index] = b_predicted_train_group_2.values
        b_predict_test.loc[b_predicted_test_group_1.index] = b_predicted_test_group_1.values
        b_predict_test.loc[b_predicted_test_group_2.index] = b_predicted_test_group_2.values    

      # this is for when we want to use model1 or model2, and not model1+model2 for each group
      else:
        b_predict_train = pd.Series([helper(a,b,c) for a,b,c in zip(which_model_train,b_predicted_train_1,b_predicted_train_2)],index=train_index)
        b_predict_test = pd.Series([helper(a,b,c) for a,b,c in zip(which_model_test,b_predicted_test_1,b_predicted_test_2)],index=test_index)

      if show:
        if method == 'svm':
          X0, X1 = b_predicted_train_1, b_predicted_train_2
          xx, yy = make_meshgrid(b_predicted_train_1, b_predicted_train_2)
          fig, sub = plt.subplots(1, 1)
          plt.subplots_adjust(wspace=0.02, hspace=0.02)
          if svm_dim == 2:
            sub.contourf(xx, yy, clf.predict(np.c_[xx.ravel(),yy.ravel()]).reshape(xx.shape), cmap=plt.cm.coolwarm, alpha=0.8)
          else:
            sub.contourf(xx, yy, clf.predict(np.c_[xx.ravel()]).reshape(xx.shape), cmap=plt.cm.coolwarm, alpha=0.8)
          plt.scatter(X0, X1, c=b_best_train, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
          plt.xlim([xx.min(), xx.max()])
          plt.ylim([yy.min(), yy.max()])
          plt.xlabel('log_model1')
          plt.ylabel('log_model2')
          plt.xticks(())
          plt.yticks(())
          plt.show()

    elif method == 'lin':
      b_predicted_train = np.column_stack((b_predicted_train_1,b_predicted_train_2))
      b_predicted_test = np.column_stack((b_predicted_test_1,b_predicted_test_2))
      b_predicted_train = np.pad(b_predicted_train,[(0,0),(0,1)],constant_values=1)
      b_predicted_test = np.pad(b_predicted_test,[(0,0),(0,1)],constant_values=1)
      U,S,VT = np.linalg.svd(b_predicted_train,full_matrices=False)
      S = np.diag(S)
      b_predicted_train_inv = VT.T @ np.linalg.inv(S) @ U.T
      x_fit = b_predicted_train_inv @ b_train
      b_predict_test = pd.Series(b_predicted_test @ x_fit,index=test_index)
      b_predict_train = pd.Series(b_predicted_train @ x_fit,index=train_index)
      if show:
        print(f'Model1 loading score = {x_fit[0]}, model2 loading score = {x_fit[1]}, intercept {x_fit[2]}')


    elif method == 'log':
      b_predicted_train = np.column_stack((b_predicted_train_1,b_predicted_train_2))
      b_predicted_test = np.column_stack((b_predicted_test_1,b_predicted_test_2))
      model = LogisticRegression(max_iter = 1000,fit_intercept=True,solver='liblinear')
      model.fit(b_predicted_train,b_train)
      b_predict_train = pd.Series(model.predict_proba(b_predicted_train)[:, 1],index=train_index)
      b_predict_test = pd.Series(model.predict_proba(b_predicted_test)[:, 1],index=test_index)
      x_fit = np.append(model.coef_[0],model.intercept_[0])
      if show:
        print(f'Model1 loading score = {x_fit[0]}, model2 loading score = {x_fit[1]}, intercept {x_fit[2]}')

    else:
      assert False
      
    b_predict_test_int = b_predict_test.apply(lambda i: np.float(i > edge)) # these are 0 or 1
    b_predict_train_int = b_predict_train.apply(lambda i: np.float(i > edge)) # these are 0 or 1
    test_accuracy = accuracy_score(b_test,b_predict_test_int)
    train_accuracy = accuracy_score(b_train,b_predict_train_int)
    test_sensitivity = sensitivity_score(b_test,b_predict_test_int)
    train_sensitivity = sensitivity_score(b_train,b_predict_train_int)
    test_specificity = specificity_score(b_test,b_predict_test_int)
    train_specificity = specificity_score(b_train,b_predict_train_int) 
    if np.isnan(test_sensitivity):
      test_sensitivity = 0 # this is if we have 0 'sample size'
    if np.isnan(train_sensitivity):
      train_sensitivity = 0
    if np.isnan(test_specificity):
      test_specificity = 0 # this is if we have 0 'sample size'
    if np.isnan(train_specificity):
      train_specificity = 0
    test_abs_differences = abs_differences(b_test,b_predict_test,test_index)
    train_abs_differences = abs_differences(b_train,b_predict_train,train_index)
    confusion = confusion_matrix(b_test,b_predict_test_int)
    test_sensitivity_sample = np.count_nonzero(b_test == 1)
    test_specificity_sample = np.count_nonzero(b_test == 0)
    train_sensitivity_sample = np.count_nonzero(b_train == 1)
    train_specificity_sample = np.count_nonzero(b_train == 0)  
    result['test_accuracy'].append(test_accuracy)
    result['train_accuracy'].append(train_accuracy)
    result['test_abs_differences'].loc[test_abs_differences.index] = test_abs_differences.values
    result['train_abs_differences'].loc[train_abs_differences.index,i] = train_abs_differences.values
    result['test_sensitivity'].append(test_sensitivity)
    result['train_sensitivity'].append(train_sensitivity)
    result['test_specificity'].append(test_specificity)
    result['train_specificity'].append(train_specificity)
    result['test_sensitivity_sample'].append(test_sensitivity_sample)
    result['test_specificity_sample'].append(test_specificity_sample)
    result['train_sensitivity_sample'].append(train_sensitivity_sample)
    result['train_specificity_sample'].append(train_specificity_sample)
    result['confusion'].append(confusion)
    result['b_train'].append(b_train)
    result['b_predicted_test'].loc[b_predict_test.index] = b_predict_test.values
    result['x_fit'].append([out_1['x_fit'],out_2['x_fit']])
  return result

In [ ]:
class NNet(nn.Module):
    def __init__(self, input_dim, hidden_layer_sizes, loss=nn.BCELoss(), sigmoid=False):
        super().__init__()
      
        self.input_dim = input_dim
        self.layer_sizes = hidden_layer_sizes
        self.iter = 0
        # The loss function could be MSE or BCELoss depending on the problem
        self.lossFct = loss

        # We leave the optimizer empty for now to assign flexibly
        self.optim = None

        hidden_layer_sizes = [input_dim] + hidden_layer_sizes
        last_layer = nn.Linear(hidden_layer_sizes[-1], 1)
        self.layers =\
            [nn.Sequential(nn.Linear(input_, output_), nn.ReLU())
            for input_, output_ in 
            zip(hidden_layer_sizes, hidden_layer_sizes[1:])] +\
            [last_layer]
        
        # The output activation depends on the problem
        if sigmoid:
            self.layers = self.layers + [nn.Sigmoid()]
            
        self.layers = nn.Sequential(*self.layers)

    def compute_l1_loss(self, w):
        return torch.abs(w).sum()

    def forward(self, x):
        x = self.layers(x)
        return x
    
    def train(self, data_loader, epochs, validation_data=None):

        for epoch in range(epochs):
            running_loss = self._train_iteration(data_loader)
            val_loss = None
            if validation_data is not None:
                y_hat = self(validation_data['X'])
                val_loss = self.lossFct(input=y_hat, target=validation_data['y']).detach().numpy()
            #     print('[%d] loss: %.3f | validation loss: %.3f' %
            #       (epoch + 1, running_loss, val_loss))
            # else:
            #     print('[%d] loss: %.3f' %
            #       (epoch + 1, running_loss))

    def _train_iteration(self,data_loader):
        running_loss = 0.0
        for i, (X,y) in enumerate(data_loader):
            
            X = X.float()
            y = y.unsqueeze(1).float()
            
            X_ = Variable(X, requires_grad=True)
            y_ = Variable(y)
              
            ### Comment out the typical gradient calculation
#             pred = self(X)
#             loss = self.lossFct(pred, y)
            
#             self.optim.zero_grad()
#             loss.backward()
            
            ### Add the closure function to calculate the gradient.
            def closure():
                if torch.is_grad_enabled():
                    self.optim.zero_grad()
                output = self(X_)
                
                loss = self.lossFct(output, y_)

                # Compute L1 loss component
                l1_weight = 0.0
                l1_parameters = []
                for parameter in self.parameters():
                    l1_parameters.append(parameter.view(-1))
                l1 = l1_weight * self.compute_l1_loss(torch.cat(l1_parameters))
                
                # Add L1 loss component
                loss += l1

                # all_linear1_params = self.layers[0].weight[0]
                # l1_regularization = 0.05 * torch.norm(all_linear1_params, 1)
                # #l2_regularization = lambda2 * torch.norm(all_linear2_params, 2)
                # loss += l1_regularization

                if loss.requires_grad:
                    loss.backward()
                return loss
            
            self.optim.step(closure)
            
            # calculate the loss again for monitoring
            output = self(X_)
            loss = closure()
            running_loss += loss.item()
              
        return running_loss
    
    # I like to include a sklearn like predict method for convenience
    def predict(self, X):
        X = torch.Tensor(X)
        return self(X).detach().numpy().squeeze()

def my_loss(output, target):
  a = -3.25
  loss = torch.log(1+torch.exp(a*(1-2*torch.abs(output-target))))
  return torch.mean(loss)

class ExperimentData(Dataset):
  def __init__(self, X, y):
      self.X = X
      self.y = y
      
  def __len__(self):
      return self.X.shape[0]
  
  def __getitem__(self, idx):
      return self.X[idx,:], self.y[idx]

def logistic_estimator_with_pytorch(A_train,A_test,b_train,b_test,test_index,train_index,r = 0,q=0,p=0,edge=0.5,l2_reg=0.0):
  data = ExperimentData(A_train,b_train)
  INPUT_SIZE = A_train.shape[1]
  EPOCHS=1 # Few epochs to avoid overfitting
  pred_train = {}
  HIDDEN_LAYER_SIZE = []
  data_loader = DataLoader(data, batch_size=A_train.shape[0])
  net = NNet(INPUT_SIZE, HIDDEN_LAYER_SIZE, sigmoid=True, loss = my_loss)
  net.optim = LBFGS(net.parameters(), history_size=10, max_iter=10)#,lr=1.5)
  net.train(data_loader, EPOCHS)
  if len(test_index)>0:
    b_predicted_test = net.predict(A_test) # these are float
  else:
    b_predicted_test = []
  if A_test.shape[0] == 1:
    b_predicted_test = [b_predicted_test]
  b_predicted_train = net.predict(A_train)
  b_predicted_test_2 = np.array([np.float(i > edge) for i in b_predicted_test]) # these are 0 or 1
  b_predicted_train_2 = np.array([np.float(i > edge) for i in b_predicted_train]) # these are 0 or 1
  test_accuracy = accuracy_score(b_test,b_predicted_test_2)
  train_accuracy = accuracy_score(b_train,b_predicted_train_2)
  test_sensitivity = sensitivity_score(b_test,b_predicted_test_2)
  train_sensitivity = sensitivity_score(b_train,b_predicted_train_2)
  test_specificity = specificity_score(b_test,b_predicted_test_2)
  train_specificity = specificity_score(b_train,b_predicted_train_2) 
  if np.isnan(test_sensitivity):
    test_sensitivity = 0 # this is if we have 0 'sample size'
  if np.isnan(train_sensitivity):
    train_sensitivity = 0
  if np.isnan(test_specificity):
    test_specificity = 0 # this is if we have 0 'sample size'
  if np.isnan(train_specificity):
    train_specificity = 0
  if np.isnan(test_accuracy):
    test_accuracy = 0 # this is if we have 0 'sample size'
  x_fit = np.append(net.layers[0].weight[0].detach().numpy(),net.layers[0].bias[0].detach().numpy()) #bias is always the last one
  test_abs_differences = abs_differences(b_test,b_predicted_test,test_index)
  train_abs_differences = abs_differences(b_train,b_predicted_train,train_index)
  #test_r2 = r2_score(b_test,b_predicted_test) this one only has 4 people in it and is no use
  confusion = confusion_matrix(b_test,b_predicted_test_2)
  result = {}
  result['test_accuracy'] = test_accuracy
  result['train_accuracy'] = train_accuracy
  result['x_fit'] = x_fit
  #result['test_r^2'] = test_r2
  result['test_abs_differences'] = test_abs_differences
  result['train_abs_differences'] = train_abs_differences
  result['test_sensitivity'] = test_sensitivity
  result['train_sensitivity'] = train_sensitivity
  result['b_predicted_test_2'] = b_predicted_test_2
  result['confusion'] = confusion
  result['test_specificity'] = test_specificity
  result['train_specificity'] = train_specificity
  result['b_predicted_test'] = b_predicted_test
  result['b_predicted_train'] = b_predicted_train
  return result

# def logistic_estimator_with_keras(A_train,A_test,b_train,b_test,test_index,train_index,r = 0,q=0,p=0,edge=0.5,l2_reg=1.0):
#   # Our vectorized labels
#   b_train = np.asarray(b_train).astype('float32').reshape((-1,1))
#   b_test = np.asarray(b_test).astype('float32').reshape((-1,1))

#   def custom_loss_function(y_true, y_pred):
#     a = -3.25
#     abc = tf.math.log(1+tf.math.exp(a*(1-2*tf.math.abs(y_true-y_pred))))
#     return tf.reduce_mean(abc, axis=-1)
#   model = keras.Sequential()
#   model.add(keras.layers.Dense(1, 
#                   #kernel_initializer='glorot_uniform',
#                   input_dim=A_train.shape[1], 
#                   #kernel_regularizer=L1L2(l1=0.0, l2=0.1),       
#                   #kernel_regularizer=keras.regularizers.l2(l2_reg),                
#                   activation='sigmoid'))

#   model.compile(optimizer='sgd',
#                 loss='binary_crossentropy',
#                 metrics=['binary_accuracy'])
  
#   model.fit(A_train, b_train, epochs=50, shuffle=True, validation_data=(A_test, b_test), verbose = 0)

#   if len(test_index)>0:
#     b_predicted_test = np.reshape(model.predict(A_test),(1, -1))[0] # these are float
#   else:
#     b_predicted_test = []
#   b_predicted_train = np.reshape(model.predict(A_train),(1, -1))[0]
#   b_predicted_test_2 = np.array([np.float(i > edge) for i in b_predicted_test]) # these are 0 or 1
#   b_predicted_train_2 = np.array([np.float(i > edge) for i in b_predicted_train]) # these are 0 or 1
#   test_accuracy = accuracy_score(b_test,b_predicted_test_2)
#   train_accuracy = accuracy_score(b_train,b_predicted_train_2)
#   test_sensitivity = sensitivity_score(b_test,b_predicted_test_2)
#   train_sensitivity = sensitivity_score(b_train,b_predicted_train_2)
#   test_specificity = specificity_score(b_test,b_predicted_test_2)
#   train_specificity = specificity_score(b_train,b_predicted_train_2) 
#   if np.isnan(test_sensitivity):
#     test_sensitivity = 0 # this is if we have 0 'sample size'
#   if np.isnan(train_sensitivity):
#     train_sensitivity = 0
#   if np.isnan(test_specificity):
#     test_specificity = 0 # this is if we have 0 'sample size'
#   if np.isnan(train_specificity):
#     train_specificity = 0
#   if np.isnan(test_accuracy):
#     test_accuracy = 0 # this is if we have 0 'sample size'
#   x_fit = np.append(model.layers[0].get_weights()[0],model.layers[0].get_weights()[1])
#   test_abs_differences = abs_differences(b_test,b_predicted_test,test_index)
#   train_abs_differences = abs_differences(b_train,b_predicted_train,train_index)
#   #test_r2 = r2_score(b_test,b_predicted_test) this one only has 4 people in it and is no use
#   confusion = confusion_matrix(b_test,b_predicted_test_2)
#   result = {}
#   result['test_accuracy'] = test_accuracy
#   result['train_accuracy'] = train_accuracy
#   result['x_fit'] = x_fit
#   #result['test_r^2'] = test_r2
#   result['test_abs_differences'] = test_abs_differences
#   result['train_abs_differences'] = train_abs_differences
#   result['test_sensitivity'] = test_sensitivity
#   result['train_sensitivity'] = train_sensitivity
#   result['b_predicted_test_2'] = b_predicted_test_2
#   result['confusion'] = confusion
#   result['test_specificity'] = test_specificity
#   result['train_specificity'] = train_specificity
#   result['b_predicted_test'] = b_predicted_test
#   result['b_predicted_train'] = b_predicted_train
#   return result

def logistic_estimator(A_train,A_test,b_train,b_test,test_index,train_index,r = 0,q=0,p=0,edge=0.5,l2_reg=1.0):
  model = LogisticRegression(max_iter = 1000,fit_intercept=True,solver='liblinear',C=l2_reg)
  model.fit(A_train,b_train)
  if len(test_index)>0:
    b_predicted_test = model.predict_proba(A_test)[:, 1] # these are float
  else:
    b_predicted_test = []
  b_predicted_train = model.predict_proba(A_train)[:, 1]
  b_predicted_test_2 = np.array([np.float(i > edge) for i in b_predicted_test]) # these are 0 or 1
  b_predicted_train_2 = np.array([np.float(i > edge) for i in b_predicted_train]) # these are 0 or 1
  test_accuracy = accuracy_score(b_test,b_predicted_test_2)
  train_accuracy = accuracy_score(b_train,b_predicted_train_2)
  test_sensitivity = sensitivity_score(b_test,b_predicted_test_2)
  train_sensitivity = sensitivity_score(b_train,b_predicted_train_2)
  test_specificity = specificity_score(b_test,b_predicted_test_2)
  train_specificity = specificity_score(b_train,b_predicted_train_2) 
  if np.isnan(test_sensitivity):
    test_sensitivity = 0 # this is if we have 0 'sample size'
  if np.isnan(train_sensitivity):
    train_sensitivity = 0
  if np.isnan(test_specificity):
    test_specificity = 0 # this is if we have 0 'sample size'
  if np.isnan(train_specificity):
    train_specificity = 0
  if np.isnan(test_accuracy):
    test_accuracy = 0 # this is if we have 0 'sample size'
  x_fit = np.append(model.coef_[0],model.intercept_[0])
  test_abs_differences = abs_differences(b_test,b_predicted_test,test_index)
  train_abs_differences = abs_differences(b_train,b_predicted_train,train_index)
  #test_r2 = r2_score(b_test,b_predicted_test) this one only has 4 people in it and is no use
  confusion = confusion_matrix(b_test,b_predicted_test_2)
  result = {}
  result['test_accuracy'] = test_accuracy
  result['train_accuracy'] = train_accuracy
  result['x_fit'] = x_fit
  #result['test_r^2'] = test_r2
  result['test_abs_differences'] = test_abs_differences
  result['train_abs_differences'] = train_abs_differences
  result['test_sensitivity'] = test_sensitivity
  result['train_sensitivity'] = train_sensitivity
  result['b_predicted_test_2'] = b_predicted_test_2
  result['confusion'] = confusion
  result['test_specificity'] = test_specificity
  result['train_specificity'] = train_specificity
  result['b_predicted_test'] = b_predicted_test
  result['b_predicted_train'] = b_predicted_train
  return result

def random_forest_estimator(A_train,A_test,b_train,b_test,test_index,train_index,r = 0,q=0,p=0,edge=0.5,l2_reg=1.0):
  model = RandomForestClassifier(max_depth=4)
  model.fit(A_train,b_train)
  if len(test_index)>0:
    b_predicted_test = model.predict_proba(A_test)[:, 1] # these are float
  else:
    b_predicted_test = []
  b_predicted_train = model.predict_proba(A_train)[:, 1]
  b_predicted_test_2 = np.array([np.float(i > edge) for i in b_predicted_test]) # these are 0 or 1
  b_predicted_train_2 = np.array([np.float(i > edge) for i in b_predicted_train]) # these are 0 or 1
  test_accuracy = accuracy_score(b_test,b_predicted_test_2)
  train_accuracy = accuracy_score(b_train,b_predicted_train_2)
  test_sensitivity = sensitivity_score(b_test,b_predicted_test_2)
  train_sensitivity = sensitivity_score(b_train,b_predicted_train_2)
  test_specificity = specificity_score(b_test,b_predicted_test_2)
  train_specificity = specificity_score(b_train,b_predicted_train_2) 
  if np.isnan(test_sensitivity):
    test_sensitivity = 0 # this is if we have 0 'sample size'
  if np.isnan(train_sensitivity):
    train_sensitivity = 0
  if np.isnan(test_specificity):
    test_specificity = 0 # this is if we have 0 'sample size'
  if np.isnan(train_specificity):
    train_specificity = 0
  if np.isnan(test_accuracy):
    test_accuracy = 0 # this is if we have 0 'sample size'
  x_fit = 0#np.append(model.coef_[0],model.intercept_[0])
  test_abs_differences = abs_differences(b_test,b_predicted_test,test_index)
  train_abs_differences = abs_differences(b_train,b_predicted_train,train_index)
  #test_r2 = r2_score(b_test,b_predicted_test) this one only has 4 people in it and is no use
  confusion = confusion_matrix(b_test,b_predicted_test_2)
  result = {}
  result['test_accuracy'] = test_accuracy
  result['train_accuracy'] = train_accuracy
  result['x_fit'] = x_fit
  #result['test_r^2'] = test_r2
  result['test_abs_differences'] = test_abs_differences
  result['train_abs_differences'] = train_abs_differences
  result['test_sensitivity'] = test_sensitivity
  result['train_sensitivity'] = train_sensitivity
  result['b_predicted_test_2'] = b_predicted_test_2
  result['confusion'] = confusion
  result['test_specificity'] = test_specificity
  result['train_specificity'] = train_specificity
  result['b_predicted_test'] = b_predicted_test
  result['b_predicted_train'] = b_predicted_train
  return result

def cross_correlate(X,y, pad, estimator, r = 0,nr_folds = 10, p = 0, q = 0,edge = 0.5,b_compare = None,edge2=None,l2_reg=1.0):
  # be careful the default estimator is 'baked in' so best not to have one. If you change it, you have to 'restart the runtime'.
  if estimator == logistic_estimator:
    pad = False # it pads automatically, you can't turn it off
  if edge2 is None:
    edge2 = edge
#non-parallel
  result = {}
  if b_compare is not None:
    result['delta_accuracy'] = []
    result['delta_r^2'] = []
    result['delta_sensitivitys'] = []   
    result['delta_specificitys'] = []  
  result['test_accuracy'] = []
  result['train_accuracy'] = []
  result['x_fit'] = []
  result['test_abs_differences'] = pd.Series(dtype='float64',index=X.index)
  result['train_abs_differences'] = pd.DataFrame(dtype='float64',index=X.index)
  result['test_sensitivity'] = []
  result['train_sensitivity'] = []
  result['test_sensitivity_sample'] = []
  result['test_specificity_sample'] = []
  result['train_sensitivity_sample'] = []
  result['train_specificity_sample'] = []
  result['confusion'] = []
  result['test_specificity'] = []
  result['train_specificity'] = []
  result['b_train'] = []
  result['b_predicted_test'] = pd.Series(dtype='float64',index=X.index)
  if estimator == estimator_SVD:
    result['S'] = []
  if nr_folds == 1:
    assert False
    # A_train, A_test, b_train, b_test = train_test_split(X,y)
    # if pad:
    #   if len(A_train.shape) == 1:
    #     A_train = pd.DataFrame(A_train)
    #     list_of_ones = pd.Series(np.ones_like(b_train),name='pad',index=A_train.index)
    #     A_train = A_train.join(list_of_ones)
    #     A_train = A_train.values
    #     A_test = pd.DataFrame(A_test)
    #     list_of_ones = pd.Series(np.ones_like(b_test),name='pad',index=A_test.index)
    #     A_test = A_test.join(list_of_ones)
    #     A_test = A_test.values
    #   else:
    #     A_train = np.pad(A_train,[(0,0),(0,1)],constant_values=1)
    #     A_test = np.pad(A_test,[(0,0),(0,1)],constant_values=1)
    # test_accuracy,train_accuracy,x_fit,test_r2,train_r2,train_sensitivity,test_sensitivity,confusion,b_predicted_test_2 = estimator(A_train,A_test,b_train,b_test,r=r,p=p,q=q,edge=edge,l2_reg=l2_reg)
    # sensitivity_sample = np.count_nonzero(b_test == 1.0)
    # result.append([test_accuracy,train_accuracy,x_fit,test_r2,train_r2,test_sensitivity,train_sensitivity,confusion,sensitivity_sample])
  else:
    kf = KFold(n_splits=nr_folds,shuffle=True)
    gen = kf.split(X)
    gens = []
    for generator in gen:
      gens.append(generator)
    i = 0
    for generator in gens:
      i+=1
      A_train, A_test = X.iloc[generator[0]].to_numpy(), X.iloc[generator[1]].to_numpy()
      test_index = X.iloc[generator[1]].index
      train_index = X.iloc[generator[0]].index
      if len(A_train.shape) == 1:
        A_train = A_train.reshape(-1,1)
        A_test = A_test.reshape(-1,1)
      b_train, b_test = np.rint(y.iloc[generator[0]].to_numpy()), np.rint(y.iloc[generator[1]].to_numpy())
      for i in b_train:
        assert((i == 1) | (i == 0))
      for i in b_test:
        assert((i == 1) | (i == 0))
      if pad:
        if len(A_train.shape) == 1:
          A_train = pd.DataFrame(A_train)
          list_of_ones = pd.Series(np.ones_like(b_train),name='pad',index=A_train.index)
          A_train = A_train.join(list_of_ones)
          A_train = A_train.values
          A_test = pd.DataFrame(A_test)
          list_of_ones = pd.Series(np.ones_like(b_test),name='pad',index=A_test.index)
          A_test = A_test.join(list_of_ones)
          A_test = A_test.values
        else:
          A_train = np.pad(A_train,[(0,0),(0,1)],constant_values=1)
          A_test = np.pad(A_test,[(0,0),(0,1)],constant_values=1)
      
      out = estimator(A_train,A_test,b_train,b_test,test_index,train_index,r=r,p=p,q=q,edge=edge,l2_reg=l2_reg)

      b_predicted_test = pd.Series(out['b_predicted_test'],index=test_index)
      test_sensitivity_sample = np.count_nonzero(b_test == 1)
      test_specificity_sample = np.count_nonzero(b_test == 0)
      train_sensitivity_sample = np.count_nonzero(b_train == 1)
      train_specificity_sample = np.count_nonzero(b_train == 0)      
      if b_compare is not None:
        _, b_compare_test = b_compare.iloc[generator[0]], b_compare.iloc[generator[1]]
        b_compare_test_2 = [int(row > edge2) for row in b_compare_test]
        compare_accuracy = accuracy_score(b_test,b_compare_test_2)
        compare_r2 = r2_score(b_test,b_compare_test)
        sensitivity_0 = np.sum([np.rint((a==b) & (c == 1)) for a,b,c in zip(b_compare_test_2,out['b_predicted_test_2'],b_test)])
        sensitivity_neg_1 = np.sum([np.rint((a>b) & (c == 1)) for a,b,c in zip(b_compare_test_2,out['b_predicted_test_2'],b_test)])
        sensitivity_pos_1 = np.sum([np.rint((a<b) & (c == 1)) for a,b,c in zip(b_compare_test_2,out['b_predicted_test_2'],b_test)])
        specificity_0 = np.sum([np.rint((a==b) & (c == 0)) for a,b,c in zip(b_compare_test_2,out['b_predicted_test_2'],b_test)])
        specificity_neg_1 = np.sum([np.rint((a<b) & (c == 0)) for a,b,c in zip(b_compare_test_2,out['b_predicted_test_2'],b_test)])
        specificity_pos_1 = np.sum([np.rint((a>b) & (c == 0)) for a,b,c in zip(b_compare_test_2,out['b_predicted_test_2'],b_test)])
        delta_r2 = out['test_r2'] - compare_r2
        delta_accuracy = out['test_accuracy'] - compare_accuracy
        result['delta_accuracy'].append(delta_accuracy)
        result['delta_r^2'].append(delta_r2)
        result['delta_sensitivitys'].append([sensitivity_0,sensitivity_neg_1,sensitivity_pos_1]) 
        result['delta_specificitys'].append([specificity_0,specificity_neg_1,specificity_pos_1]) 
      result['test_accuracy'].append(out['test_accuracy'])
      result['train_accuracy'].append(out['train_accuracy'])
      result['x_fit'].append(out['x_fit'])
      result['test_abs_differences'].loc[out['test_abs_differences'].index] = out['test_abs_differences'].values
      result['train_abs_differences'].loc[out['train_abs_differences'].index,i] = out['train_abs_differences'].values
      result['test_sensitivity'].append(out['test_sensitivity'])
      result['train_sensitivity'].append(out['train_sensitivity'])
      result['test_specificity'].append(out['test_specificity'])
      result['train_specificity'].append(out['train_specificity'])
      result['test_sensitivity_sample'].append(test_sensitivity_sample)
      result['test_specificity_sample'].append(test_specificity_sample)
      result['train_sensitivity_sample'].append(train_sensitivity_sample)
      result['train_specificity_sample'].append(train_specificity_sample)
      result['confusion'].append(out['confusion'])
      result['b_train'].append(b_train)
      result['b_predicted_test'].loc[b_predicted_test.index] = b_predicted_test.values
      if estimator == estimator_SVD:
        result['S'].append(out['S'])
  return result

def r_estimator(A_train,A_test,b_train,b_test,r,q,p,edge=0.5):
  ny = A_train.shape[1]
  P = np.random.randn(ny,r+p)
  Z = A_train @ P
  for k in range(q):
    Z = A_train @ (A_train.T @ Z)
  Q,R = np.linalg.qr(Z)
  Y = Q.T @ A_train
  UY,S,VT = np.linalg.svd(Y,full_matrices=False)
  U = Q @ UY
  S = np.diag(S)
  A_train_inv = VT.T @ np.linalg.inv(S) @ U.T
  x_fit = A_train_inv @ b_train
  b_predicted_test = A_test @ x_fit
  b_predicted_test_2 = [int(row > edge) for row in b_predicted_test]
  b_predicted_train = A_train @ x_fit
  b_predicted_train = [int(row > edge) for row in b_predicted_train]
  test_accuracy = accuracy_score(b_test,b_predicted_test_2)
  train_accuracy = accuracy_score(b_train,b_predicted_train_2)
  test_sensitivity = sensitivity_score(b_test,b_predicted_test_2)
  train_sensitivity = sensitivity_score(b_train,b_predicted_train_2)
  test_specificity = specificity_score(b_test,b_predicted_test_2)
  train_specificity = specificity_score(b_train,b_predicted_train_2) 
  if np.isnan(test_sensitivity):
    test_sensitivity = 0 # this is if we have 0 'sample size'
  if np.isnan(train_sensitivity):
    train_sensitivity = 0
  if np.isnan(test_specificity):
    test_specificity = 0 # this is if we have 0 'sample size'
  if np.isnan(train_specificity):
    train_specificity = 0
  test_abs_differences = abs_differences(b_test,b_predicted_test)
  train_abs_differences = abs_differences(b_train,b_predicted_train)
  confusion = confusion_matrix(b_test,b_predicted_test_2)
  result = {}
  result['test_accuracy'] = test_accuracy
  result['train_accuracy'] = train_accuracy
  result['x_fit'] = x_fit
  #result['test_r^2'] = test_r2 this one only has 4 people in it and is no use
  result['test_abs_differences'] = test_abs_differences
  result['train_abs_differences'] = train_abs_differences
  result['test_sensitivity'] = test_sensitivity
  result['train_sensitivity'] = train_sensitivity
  result['b_predicted_test_2'] = b_predicted_test_2
  result['S'] = S
  result['confusion'] = confusion
  result['test_specificity'] = test_specificity
  result['train_specificity'] = train_specificity
  result['b_predicted_test'] = b_predicted_test
  return result

def logistic_estimator_with_k_means(A_train,A_test,b_train,b_test,test_index,train_index,r = 0,q=0,p=0,edge=0.5,l2_reg=1.0):
  kmeans = KMeans(n_clusters=2, init='k-means++', max_iter=1000, n_init=10)
  kmeans.fit(A_train)
  A_train_1 = [A_train[point] for point in range(len(A_train)) if kmeans.labels_[point] == 0]
  b_train_1 = [b_train[point] for point in range(len(b_train)) if kmeans.labels_[point] == 0]
  A_train_2 = [A_train[point] for point in range(len(A_train)) if kmeans.labels_[point] == 1]
  b_train_2 = [b_train[point] for point in range(len(b_train)) if kmeans.labels_[point] == 1]
  model1 = LogisticRegression(max_iter = 1000,fit_intercept=True,solver='liblinear',C=l2_reg)
  model1.fit(A_train_1,b_train_1)
  model2 = LogisticRegression(max_iter = 1000,fit_intercept=True,solver='liblinear',C=l2_reg)
  model2.fit(A_train_1,b_train_1)
  abc = kmeans.predict(A_test)
  A_test_1 = [A_test[point] for point in range(len(A_test)) if abc[point] == 0]
  b_test_1 = [b_test[point] for point in range(len(b_test)) if abc[point] == 0]
  A_test_2 = [A_test[point] for point in range(len(A_test)) if abc[point] == 1]
  b_test_2 = [b_test[point] for point in range(len(b_test)) if abc[point] == 1]  
  if len(test_index)>0:
    b_predicted_test_1 = model1.predict_proba(A_test_1)[:, 1] # these are float
  else:
    b_predicted_test = []
  if len(test_index)>0:
    b_predicted_test_2 = model2.predict_proba(A_test_2)[:, 1] # these are float
  else:
    b_predicted_test = []
  b_predicted_test = np.append(b_predicted_test_1,b_predicted_test_2)
  b_test = np.append(b_test_1,b_test_2)
  b_predicted_train1 = model1.predict_proba(A_train_1)[:, 1]
  b_predicted_train2 = model2.predict_proba(A_train_2)[:, 1]
  b_predicted_train = np.append(b_predicted_train1,b_predicted_train2)
  b_train = np.append(b_train_1,b_train_2)
  b_predicted_test_2 = np.array([np.float(i > edge) for i in b_predicted_test]) # these are 0 or 1
  b_predicted_train_2 = np.array([np.float(i > edge) for i in b_predicted_train]) # these are 0 or 1
  test_accuracy = accuracy_score(b_test,b_predicted_test_2)
  train_accuracy = accuracy_score(b_train,b_predicted_train_2)
  test_sensitivity = sensitivity_score(b_test,b_predicted_test_2)
  train_sensitivity = sensitivity_score(b_train,b_predicted_train_2)
  test_specificity = specificity_score(b_test,b_predicted_test_2)
  train_specificity = specificity_score(b_train,b_predicted_train_2) 
  if np.isnan(test_sensitivity):
    test_sensitivity = 0 # this is if we have 0 'sample size'
  if np.isnan(train_sensitivity):
    train_sensitivity = 0
  if np.isnan(test_specificity):
    test_specificity = 0 # this is if we have 0 'sample size'
  if np.isnan(train_specificity):
    train_specificity = 0
  if np.isnan(test_accuracy):
    test_accuracy = 0 # this is if we have 0 'sample size'
  x_fit = 0#np.append(model.coef_[0],model.intercept_[0])
  test_abs_differences = abs_differences(b_test,b_predicted_test,test_index)
  train_abs_differences = abs_differences(b_train,b_predicted_train,train_index)
  #test_r2 = r2_score(b_test,b_predicted_test) this one only has 4 people in it and is no use
  confusion = confusion_matrix(b_test,b_predicted_test_2)
  result = {}
  result['test_accuracy'] = test_accuracy
  result['train_accuracy'] = train_accuracy
  result['x_fit'] = x_fit
  #result['test_r^2'] = test_r2
  result['test_abs_differences'] = test_abs_differences
  result['train_abs_differences'] = train_abs_differences
  result['test_sensitivity'] = test_sensitivity
  result['train_sensitivity'] = train_sensitivity
  result['b_predicted_test_2'] = b_predicted_test_2
  result['confusion'] = confusion
  result['test_specificity'] = test_specificity
  result['train_specificity'] = train_specificity
  result['b_predicted_test'] = b_predicted_test
  result['b_predicted_train'] = b_predicted_train
  return result

## Putting everything together and metrics

In [ ]:
def loading_scores(X,y,pad,estimator,features1 = None, features2 = None,method = None, r=0,p=0,q=0,nr_folds=None,edge=0.5,b_compare = None, show = True, n_rep = 1,edge2=None,c=None,gamma=None,svd_fit=False,svm_dim=2,l2_reg=1.0,scores=True):
  temp = []
  temp2 = []
  temp3 = []
  temp4 = []
  temp5 = []
  temp6 = []
  temp7 = []
  temp8 = []
  temp9 = []
  temp10 = []
  temp11 = []
  temp12 = []
  temp13 = []
  temp14 =[]
  temp15 = []
  temp20 =[]
  temp21 = []
  temp24 = []
  temp25 = []
  temp26 = []
  temp27 = []
  temp28 = []
  temp29 = []
  temp32 = []
  if b_compare is not None:
    temp16 = []
    temp17 = []
    temp18 = []
    temp19 = []
    temp22 = []
    temp23 = []
    temp30 = []
    temp31 = []
  if (features1 is not None) and (features2 is None):
    X_copy = copy.deepcopy(X[features1])
  else:
    X_copy = copy.deepcopy(X)
    if (features1 is None) and estimator != estimator_models:
      features1 = X_copy.columns.to_list()
  if nr_folds is None:
    nr_folds = len(y)
  scaled_data = preprocessing.scale(X_copy)
  if estimator == estimator_models:
    X_copy = pd.Series(data=scaled_data, index = X_copy.index)
    assert(not pad)
    result = estimator_models(X,y,X.index,edge=edge,b_compare=b_compare,edge2=edge2)
    accuracy = result['accuracy']
    accuracy_err = result['accuracy_err']
    r2 = result['r2']
    r2_err = result['r2_err']
    sensitivity = result['sensitivity']
    sensitivity_err = result['sensitivity_err']
    specificity = result['specificity']
    specificity_err = result['specificity_err']
    confusion = result['confusion']
    auc = result['auc']
    #individual_std = result['individual_std'] #if we say 0.85, what is the std of that (maybe np.sqrt(0.85*0.15))
    if b_compare is not None:
      delta_accuracy = result['delta_accuracy']
      delta_accuracy_err = result['delta_accuracy_err']
      delta_r2 = result['delta_r^2']
      [delta_sensitivity_0,delta_sensitivity_neg_1,delta_sensitivity_pos_1] = result['delta_sensitivitys']
      [delta_specificity_0,delta_specificity_neg_1,delta_specificity_pos_1] = result['delta_specificitys']
      delta_sensitivity = (delta_sensitivity_pos_1 - delta_sensitivity_neg_1)/result['sensitivity_sample']
      delta_specificity = (delta_specificity_pos_1 - delta_specificity_neg_1)/result['specificity_sample']
      delta_sensitivity_err = np.sqrt(((-1-delta_sensitivity)**2*delta_sensitivity_neg_1+(delta_sensitivity)**2*delta_sensitivity_0+(1-delta_sensitivity)**2*delta_sensitivity_pos_1))/result['sensitivity_sample']
      delta_specificity_err = np.sqrt(((-1-delta_specificity)**2*delta_specificity_neg_1+(delta_specificity)**2*delta_specificity_0+(1-delta_specificity)**2*delta_specificity_pos_1))/result['specificity_sample']

    if show:
      print(f'Accuracy (test)= {np.round(accuracy,4)}+-{np.round(accuracy_err,4)}')
      print(f'R^2 (test)= {np.round(r2,4)}+-{np.round(r2_err,4)}')
      print(f'Sensitivity (test) = {np.round(sensitivity,4)}+-{np.round(sensitivity_err,4)}')
      print(f'Specificity (test) = {np.round(specificity,4)}+-{np.round(specificity_err,4)}')
      print(f'AUC = {np.round(auc,4)}')

      group_names = ["True Pos","False Pos","False Neg","True Neg"]
      group_counts = ["{0:0.0f}".format(value) for value in
                      confusion.flatten()]
      group_percentages = ["{0:.2%}".format(value) for value in
                          confusion.flatten()/np.sum(confusion)]
      group_errors = ["{0:.2%}".format(value) for value in
                          confusion.flatten()/np.sum(confusion)]
      labels = [f"{v1}\n{v2}+-{v3}" for v1, v2, v3 in
                zip(group_names,group_percentages,group_errors)]
      labels = np.asarray(labels).reshape(2,2)
      sns.heatmap(confusion, annot=labels, fmt="", cmap='Blues')
    numbers = {'Accuracy':accuracy,
             'Accuracy_Error':accuracy_err,
             'Sensitivity':sensitivity,
             'Sensitivity_Error':sensitivity_err,
             'Specificity':specificity,
             'Specificity_Error':specificity_err,
             'R^2':r2,
             'R^2_Error':r2_err,
             'AUC':auc
             }
    if b_compare is not None:
      temp = {'Delta_Accuracy': delta_accuracy,
              'Delta_Accuracy_Error': delta_accuracy_err,
              'Delta_R^2': delta_r2,
              'Delta_Sensitivity': delta_sensitivity,
              'Delta_Sensitivity_Error': delta_sensitivity_err,
              'Delta_Specificity': delta_specificity,
              'Delta_Specificity_Error': delta_specificity_err}
      numbers.update(temp)
    return _,numbers
  else:
    X_copy = pd.DataFrame(data=scaled_data, columns=X_copy.columns, index = X_copy.index)
    for i in range(n_rep):
      if features2 is None:
        result = cross_correlate(X_copy,y,pad,nr_folds=nr_folds,estimator=estimator,r=r,p=p,q=q,edge=edge,b_compare=b_compare,edge2=edge2,l2_reg=l2_reg)
      elif features1 is None:
        assert False
      else:
        result = cross_correlate_with_two_features(X=X_copy,y=y,features1=features1,features2=features2,pad=pad,estimator=estimator,method=method,nr_folds=nr_folds,edge=edge,c=c,gamma=gamma,svd_fit=svd_fit,svm_dim=svm_dim)
      predicted_test = result['b_predicted_test']
      test_mean = np.mean(result['test_accuracy'])
      test_err_mean = np.std(result['test_accuracy'])/np.sqrt(nr_folds)
      train_mean = np.mean(result['train_accuracy'])
      train_err_mean = np.std(result['train_accuracy'])/np.sqrt(nr_folds)

      def likelihood_error(error,a):
        return np.log(1+np.exp(a*(1-2*error)))
      def saturated_error(a):
        return likelihood_error(0,a)
      a = -3.25

      test_abs_differences_sum = np.sum([likelihood_error(i,a) for i in result['test_abs_differences']])
      test_abs_differences_err_sum = np.std([likelihood_error(i,a) for i in result['test_abs_differences']])*np.sqrt(len(result['test_abs_differences']))
      train_abs_differences_each = np.mean(result['train_abs_differences'],axis=1)
      train_abs_differences_each_err = np.std(result['train_abs_differences'],axis=1)/np.sqrt(nr_folds)
      train_abs_differences_sum = np.sum([likelihood_error(i,a) for i in train_abs_differences_each])

      def df(error,a):
        return -likelihood_error(error,a)**2*np.exp(a*(1-2*error))*a*(-2.0)

      train_abs_differences_err_sum = np.sqrt(np.sum([np.abs(i*df(j,a)) for i,j in zip(train_abs_differences_each_err,train_abs_differences_each)]))
      # this is really not the same likelihood function that you can use for y_abs_difference, but usually this is what people do. In other words, if the error for a line is Gaussian, the error for just y_pred = y_mean may not be Gaussian but you still do the 'Sum of Squares' for R^2.
      y_abs_differences = [likelihood_error(np.abs(i-np.mean(y)),a) for i in y]
      y_abs_differences_sum = np.sum(y_abs_differences)
      y_abs_differences_err_sum = np.std(y_abs_differences)*np.sqrt(len(y_abs_differences))
      saturated_model = [saturated_error(a) for i in y]
      test_r2 = (test_abs_differences_sum-y_abs_differences_sum)/(saturated_model-y_abs_differences_sum)
      test_r2_err = np.sqrt(1.0/(saturated_model-y_abs_differences_sum)**2*test_abs_differences_err_sum**2+((test_abs_differences_sum-saturated_model)/(saturated_model-y_abs_differences_sum)**2)**2*y_abs_differences_err_sum**2)
      train_r2 = (train_abs_differences_sum-y_abs_differences_sum)/(saturated_model-y_abs_differences_sum)
      train_r2_err = np.sqrt(1.0/(saturated_model-y_abs_differences_sum)**2*train_abs_differences_err_sum**2+((train_abs_differences_sum-saturated_model)/(saturated_model-y_abs_differences_sum)**2)**2*y_abs_differences_err_sum**2)
      true_positives_test = np.sum([i*j for i,j in zip(result['test_sensitivity'],result['test_sensitivity_sample'])]) # here we need to go back to 1's and 0's because for sensitivity, each fold has different 'sample size'
      false_negatives_test = np.sum([(1-i)*j for i,j in zip(result['test_sensitivity'],result['test_sensitivity_sample'])])
      test_sensitivity = true_positives_test/(false_negatives_test+true_positives_test) 
      test_sensitivity_err = np.sqrt(test_sensitivity*(1-test_sensitivity)/np.sum(result['test_sensitivity_sample']))
      true_positives_train = np.sum([i*j for i,j in zip(result['train_sensitivity'],result['train_sensitivity_sample'])]) # here we need to go back to 1's and 0's because for sensitivity, each fold has different 'sample size'
      false_negatives_train = np.sum([(1-i)*j for i,j in zip(result['train_sensitivity'],result['train_sensitivity_sample'])])
      train_sensitivity = true_positives_train/(false_negatives_train+true_positives_train) 
      train_sensitivity_err = np.sqrt(train_sensitivity*(1-train_sensitivity)/np.sum(result['train_sensitivity_sample']))
      true_negatives_test = np.sum([i*j for i,j in zip(result['test_specificity'],result['test_specificity_sample'])]) # here we need to go back to 1's and 0's because for sensitivity, each fold has different 'sample size'
      false_positives_test = np.sum([(1-i)*j for i,j in zip(result['test_specificity'],result['test_specificity_sample'])])
      test_specificity = true_negatives_test/(false_positives_test+true_negatives_test) 
      test_specificity_err = np.sqrt(test_specificity*(1-test_specificity)/np.sum(result['test_specificity_sample']))
      true_negatives_train = np.sum([i*j for i,j in zip(result['train_specificity'],result['train_specificity_sample'])]) # here we need to go back to 1's and 0's because for sensitivity, each fold has different 'sample size'
      false_positives_train = np.sum([(1-i)*j for i,j in zip(result['train_specificity'],result['train_specificity_sample'])])
      train_specificity = true_negatives_train/(false_positives_train+true_negatives_train) 
      train_specificity_err = np.sqrt(train_specificity*(1-train_specificity)/np.sum(result['train_specificity_sample']))
      confusion = np.mean(result['confusion'],axis=0)
      confusion_err = np.std(result['confusion'],axis=0)/np.sqrt(nr_folds)
      if b_compare is not None:
        delta_accuracy = np.mean(result['delta_accuracy'])
        delta_accuracy_err = np.std(result['delta_accuracy'])/np.sqrt(nr_folds)
        delta_r2 = np.mean(result['delta_r^2'])
        delta_r2_err = np.std(result['delta_r^2'])/np.sqrt(nr_folds)
        delta_sensitivity_0 = np.sum([i[0] for i in result['delta_sensitivitys']])
        delta_sensitivity_neg1 = np.sum([i[1] for i in result['delta_sensitivitys']])
        delta_sensitivity_pos1 = np.sum([i[2] for i in result['delta_sensitivitys']])
        delta_sensitivity = (delta_sensitivity_pos1 - delta_sensitivity_neg1)/(false_negatives_test+true_positives_test)
        delta_sensitivity_err = np.sqrt(((-1-delta_sensitivity)**2*delta_sensitivity_neg1+(delta_sensitivity)**2*delta_sensitivity_0+(1-delta_sensitivity)**2*delta_sensitivity_pos1))/(false_negatives_test+true_positives_test)
        delta_specificity_0 = np.sum([i[0] for i in result['delta_specificitys']])
        delta_specificity_neg1 = np.sum([i[1] for i in result['delta_specificitys']])
        delta_specificity_pos1 = np.sum([i[2] for i in result['delta_specificitys']])
        delta_specificity = (delta_specificity_pos1 - delta_specificity_neg1)/(false_positives_test+true_negatives_test)
        delta_specificity_err = np.sqrt(((-1-delta_specificity)**2*delta_specificity_neg1+(delta_specificity)**2*delta_specificity_0+(1-delta_specificity)**2*delta_specificity_pos1))/(false_positives_test+true_negatives_test)
      if (features2 is None) & (scores == True):
        if (len(X_copy.shape) == 1) and (estimator!=logistic_estimator) and (not pad):
          importance = [1.0]
          importance_stdev = [0.0]
          average = 0.0
        else:
          importance = np.mean(result['x_fit'],axis=0)
          importance_stdev = np.std(result['x_fit'],axis=0)[:-1]
          average = importance[-1]
          importance = importance[:-1]
        temp9.append(average)
        loading = [i*np.std(X[j]) for i,j in zip(importance,features1)]
        loading_stdev = [i*np.std(X[j]) for i,j in zip(importance_stdev,features1)]
        average_loading = average - np.sum([i*np.mean(X[j]) for i,j in zip(loading,features1)])
      else:
        loading,loading_stdev,importance,importance_stdev,average = [1.0],[0.0],[1.0],[0.0],0
      temp.append(test_mean)
      temp2.append(test_err_mean)
      temp3.append(train_mean)
      temp4.append(train_err_mean)
      temp5.append(loading)
      temp6.append(loading_stdev)
      temp7.append(importance)
      temp8.append(importance_stdev)
      temp10.append(test_r2)
      temp11.append(test_r2_err)
      temp12.append(train_r2)
      temp13.append(train_r2_err)
      temp14.append(confusion)
      temp15.append(confusion_err)
      temp20.append(test_sensitivity)
      temp21.append(test_sensitivity_err)
      temp24.append(train_sensitivity)
      temp25.append(train_sensitivity_err)
      temp26.append(test_specificity)
      temp27.append(test_specificity_err)
      temp28.append(train_specificity)
      temp29.append(train_specificity_err)
      temp32.append(predicted_test)    
      if b_compare is not None:
        temp16.append(delta_accuracy)
        temp17.append(delta_accuracy_err)
        temp18.append(delta_r2)
        temp19.append(delta_r2_err)
        temp22.append(delta_sensitivity)
        temp23.append(delta_sensitivity_err)
        temp30.append(delta_specificity)
        temp31.append(delta_specificity_err)
    predictions = pd.concat(temp32, axis=1, sort=False).mean(axis=1)
    auc = roc_auc_score(y,predictions)
    mean_test = (np.mean(temp))
    mean_err_test = (np.mean(temp2))
    mean_train = (np.mean(temp3))
    mean_err_train = (np.mean(temp4))
    r2_test = (np.mean(temp10))
    r2_err_test = (np.mean(temp11))
    r2_train = (np.mean(temp12))
    r2_err_train = (np.mean(temp13))
    cf_matrix = (np.mean(temp14,axis=0))
    cf_matrix_err = (np.mean(temp15,axis=0))
    sensitivity_test = (np.mean(temp20))
    sensitivity_err_test = (np.mean(temp21))
    sensitivity_train = (np.mean(temp24))
    sensitivity_err_train = (np.mean(temp25))
    specificity_test = (np.mean(temp26))
    specificity_err_test = (np.mean(temp27))
    specificity_train = (np.mean(temp28))
    specificity_err_train = (np.mean(temp29))
    if b_compare is not None:
      mean_delta_accuracy = (np.mean(temp16))
      mean_err_delta_accuracy = (np.mean(temp17))
      mean_delta_r2 = (np.mean(temp18))
      mean_err_delta_r2 = (np.mean(temp19))
      mean_delta_sensitivity = (np.mean(temp22))
      mean_err_delta_sensitivity = (np.mean(temp23))
      mean_delta_specificity = (np.mean(temp30))
      mean_err_delta_specificity = (np.mean(temp31))
    if (len(X_copy.shape) == 1) or features2 is not None:
      loading_scores = [np.mean([loading[0] for loading in temp5])]
      loading_stdev = [np.mean([loading_stdev[0] for loading_stdev in temp6])]
      importance_scores = [np.mean([importance[0] for importance in temp7])]
      importance_stdev = [np.mean([importance_stdev[0] for importance_stdev in temp8])]
    else:
      loading_scores = [np.mean([loading[i] for loading in temp5]) for i in range(len(X_copy.columns))]
      loading_stdev = [np.mean([loading_stdev[i] for loading_stdev in temp6]) for i in range(len(X_copy.columns))]
      importance_scores = [np.mean([importance[i] for importance in temp7]) for i in range(len(X_copy.columns))]
      importance_stdev = [np.mean([importance_stdev[i] for importance_stdev in temp8]) for i in range(len(X_copy.columns))]
    average = (np.mean(temp9))

    if show:
      print(f'Accuracy (test)= {np.round(mean_test,4)}+-{np.round(mean_err_test,4)}')
      print(f'Acuracy (train)= {np.round(mean_train,4)}+-{np.round(mean_err_train,4)}')
      print(f'R^2 (test)= {np.round(r2_test,4)}+-{np.round(r2_err_test,4)}')
      print(f'R^2 (train)= {np.round(r2_train,4)}+-{np.round(r2_err_train,4)}')
      print(f'Sensitivity (test) = {np.round(sensitivity_test,4)}+-{np.round(sensitivity_err_test,4)}')
      print(f'Sensitivity (train) = {np.round(sensitivity_train,4)}+-{np.round(sensitivity_err_train,4)}')
      print(f'Specificity (test) = {np.round(specificity_test,4)}+-{np.round(specificity_err_test,4)}')
      print(f'Specificity (train) = {np.round(specificity_train,4)}+-{np.round(specificity_err_train,4)}')    
      print(f'AUC = {np.round(auc,4)}')

      if b_compare is not None:
        print('###Delta_accuracy and Delta_R^2###')
        print("Delta = Original model minus b_compare model")
        print(f'Delta Accuracy (test)= {np.round(mean_delta_accuracy,4)}+-{np.round(mean_err_delta_accuracy,4)}')
        print(f'Delta R^2 (test) = {np.round(mean_delta_r2,4)}+-{np.round(mean_err_delta_r2,4)}')
        print(f'Delta Sensitivity (test) = {np.round(mean_delta_sensitivity,4)}+-{np.round(mean_err_delta_sensitivity,4)}')
        print(f'Delta Specificity (test) = {np.round(mean_delta_specificity,4)}+-{np.round(mean_err_delta_specificity,4)}')
      if (len(X_copy.shape) != 1) and (features2 is None):
        plt.rcParams['figure.figsize'] = [16,8]

        x_tick = X_copy.columns.to_list()
        fig = plt.figure()
        plt.bar(x_tick,importance_scores,yerr=importance_stdev)
        plt.ylabel("Significance")
        plt.show()

      group_names = ["True Pos","False Pos","False Neg","True Neg"]
      group_counts = ["{0:0.0f}".format(value) for value in
                      cf_matrix.flatten()]
      group_percentages = ["{0:.2%}".format(value) for value in
                          cf_matrix.flatten()/np.sum(cf_matrix)]
      group_errors = ["{0:.2%}".format(value) for value in
                          cf_matrix_err.flatten()/np.sum(cf_matrix)]
      labels = [f"{v1}\n{v2}+-{v3}" for v1, v2, v3 in
                zip(group_names,group_percentages,group_errors)]
      labels = np.asarray(labels).reshape(2,2)
      sns.heatmap(cf_matrix, annot=labels, fmt="", cmap='Blues')
    numbers = {'Accuracy':mean_test,
              'Accuracy_Error':mean_err_test,
              'Sensitivity':sensitivity_test,
              'Sensitivity_Error':sensitivity_err_test,
              'Specificity':specificity_test,
              'Specificity_Error':specificity_err_test,
              'R^2':r2_test,
              'R^2_Error':r2_err_test,
              'Importance_Scores':importance_scores,
              'Average':average,
              'AUC': auc
              }
    if b_compare is not None:
      temp = {'Delta_Accuracy':mean_delta_accuracy,
              'Delta_Accuracy_Error':mean_err_delta_accuracy,
              'Delta_R^2':mean_delta_r2,
              'Delta_R^2_Error':mean_err_delta_r2,
              'Delta_Sensitivity':mean_delta_sensitivity,
              'Delta_Sensitivity_Error':mean_err_delta_sensitivity,
              'Delta_Specificity':mean_delta_specificity,
              'Delta_Specificity_Error':mean_err_delta_specificity}
      numbers.update(temp)
    return predictions,numbers



## Feature selection

In [ ]:
ca_list = ['ca_0.0','ca_1.0','ca_2.0','ca_3.0']
thal_list = ['thal_3.0','thal_6.0','thal_7.0']
slope_list = ['slope_1.0','slope_2.0','slope_3.0']
cp_list = ['cp_1.0','cp_2.0','cp_3.0','cp_4.0']
restecg_list = ['restecg_0.0','restecg_1.0','restecg_2.0']

#helper for feature_selection
def complete_cats(cat,dropped,complete,orthogonal_to,importance,importance_stdev,average,x_tick,df,X,debug):
  if complete == True:
    n = []
    pos=[]
    n_dropped = len(df.loc[df[cat] == dropped])
    for i in globals()[cat+'_list']:
      if i != cat + "_" + dropped: 
        n.append(len(df.loc[df[cat] == float(i.split("_")[1])]))
        pos.append(X.columns.get_loc(i))
    x = np.sum([n[i]*importance[pos[i]] for i in range(len(n))])/(np.sum(n)+n_dropped)
    dropped_error = np.sqrt(np.sum([n[i]*importance_stdev[pos[i]]**2 for i in range(len(n))])/len(n)*(len(n)+1)/(np.sum(n)+n_dropped))
    importance = np.append(importance,-x)
    importance_stdev = np.append(importance_stdev,dropped_error)
    for i in range(len(n)):
      importance[pos[i]] += -x
      importance_stdev[pos[i]] = np.sqrt((n[i]*importance_stdev[pos[i]]**2 + n_dropped*dropped_error**2)/(1/2*(n_dropped+n[i])))
    x_tick.append(cat+"_" + dropped)
    if debug:
      plt.figure(figsize=(22,11))
      plt.bar(x_tick,importance,yerr=importance_stdev)
      plt.ylabel("Significance")
      plt.show()
    average = average + x
  elif len(orthogonal_to) > 0:
    n = []
    pos=[]
    orthogonal = []
    for i in globals()[cat+'_list']:
      if i in X.columns.to_list(): 
        n.append(len(df.loc[df[cat] == float(i.split("_")[1])]))
        pos.append(X.columns.get_loc(i))
      elif i in orthogonal_to:
        orthogonal.append(i)
    assert (len(orthogonal) == len(globals()[cat+'_list']) - 1) and (len(n) == 1) == False
    if ((len(n) + len(orthogonal) == len(globals()[cat+'_list'])) and (len(n)>1)):
      x = np.sum([n[i]*importance[pos[i]] for i in range(len(n))])/np.sum(n)
      for i in range(len(n)):
        importance[pos[i]] += -x
      if debug:
        plt.figure(figsize=(22,11))
        plt.bar(x_tick,importance,yerr=importance_stdev)
        plt.ylabel("Significance")
        plt.show()
      average = average + x # this may need to be changed but is not used
  return importance, importance_stdev,average,x_tick

#feature-selection
not_categorical = ['age','chol','oldpeak','trestbps','thalach']
def automated_leave_k_out(X, y, target_features = False,method = 'forward_selection',metric = 'auc',usage_map=False,intermediate_n = False,gridsearch=False,size_test = 5,size_gridsearch_test=5,gridsearch_metric = 'auc', no_cv=False,keep_going = False,gridsearch_mode = False,or_features=False,decider_misses = 5,decider_type = 'history',decider_min_jumps = 5):
  assert (metric == 'auc') or (metric == 'accuracy')
  assert (gridsearch_metric == 'auc') or (gridsearch_metric == 'accuracy') or (gridsearch_metric == 'binomial_fraying')
  assert (method == 'forward_selection') or (method == 'backward_selection') or (method == 'step_forward_selection')
  assert (not (target_features and gridsearch))

  def find_next(chosen,feature,metric,generator,X,y):
    features = np.append(chosen,feature)
    assert len(features)>len(chosen)
    # For logistic regression, we don't need to take out one of every ohe feature.
    A_train, A_test = X[features].iloc[generator[0]].to_numpy(), X[features].iloc[generator[1]].to_numpy()
    b_train, b_test = np.rint(y.iloc[generator[0]].to_numpy()), np.rint(y.iloc[generator[1]].to_numpy())
    for i in b_train:
      assert((i == 1) | (i == 0))
    for i in b_test:
      assert((i == 1) | (i == 0))
    model = LogisticRegression(max_iter = 1000,fit_intercept=True,solver='liblinear',C=1000)
    model.fit(A_train,b_train)
    intercept = model.intercept_[0]
    x_fit = []
    coef_ = []
    for j in range(len(features)):
      coef_.append(model.coef_[0][j])
      x_fit.append(features[j])
    x_fit = [fit for _,fit in sorted(zip([abs(i) for i in coef_],x_fit),reverse=True)]
    if metric == 'accuracy':
      result = accuracy_score(b_train,np.rint(model.predict(A_train)))
    elif metric == 'auc':
      result = roc_auc_score(b_train,model.predict_proba(A_train)[:,1])
    test_accuracy = accuracy_score(b_test,np.rint(model.predict(A_test)))
    test_size = len(b_test)
    return result,x_fit[:len(chosen)+1],test_accuracy*test_size,model.predict_proba(A_test)[:, 1]

  def remove_next(chosen,metric,generator,X,y):
    features = chosen
    # For logistic regression, we don't need to take out one of every ohe feature.
    A_train, A_test = X[features].iloc[generator[0]].to_numpy(), X[features].iloc[generator[1]].to_numpy()
    b_train, b_test = np.rint(y.iloc[generator[0]].to_numpy()), np.rint(y.iloc[generator[1]].to_numpy())
    for i in b_train:
      assert((i == 1) | (i == 0))
    for i in b_test:
      assert((i == 1) | (i == 0))
    model = LogisticRegression(max_iter = 1000,fit_intercept=True,solver='liblinear',C=1000)
    model.fit(A_train,b_train)
    x_fit = []
    coef_ = []
    for j in range(len(features)):
      coef_.append(model.coef_[0][j])
      x_fit.append(features[j])
    x_fit = [fit for _,fit in sorted(zip([abs(i) for i in coef_],x_fit),reverse=True)]
    chosen = x_fit[:len(chosen)-1]
    features = chosen
    # For logistic regression, we don't need to take out one of every ohe feature.
    A_train, A_test = X[features].iloc[generator[0]].to_numpy(), X[features].iloc[generator[1]].to_numpy()
    b_train, b_test = np.rint(y.iloc[generator[0]].to_numpy()), np.rint(y.iloc[generator[1]].to_numpy())
    model.fit(A_train,b_train)
    intercept = model.intercept_[0]
    x_fit = []
    coef_ = []
    for j in range(len(features)):
      coef_.append(model.coef_[0][j])
      x_fit.append(features[j])
    if metric == 'accuracy':
      result = accuracy_score(b_train,np.rint(model.predict(A_train)))
    elif metric == 'auc':
      result = roc_auc_score(b_train,model.predict_proba(A_train)[:,1])
    test_accuracy = accuracy_score(b_test,np.rint(model.predict(A_test)))
    test_size = len(b_test)
    return result,x_fit,test_accuracy*test_size,model.predict_proba(A_test)[:, 1]

  #X_copy = copy.deepcopy(pd.DataFrame(preprocessing.scale(X),columns = X.columns,index = X.index))
  X_copy = copy.deepcopy(X)
  how_many = []
  chosen_all = []
  out = pd.Series(index = y.index,dtype='float64')
  test_rights = []
  if intermediate_n:
    chosen_all_intermediate = {}
    test_rights_intermediate = {}
    out_intermediate = pd.DataFrame(index = y.index,dtype='float64')
  kf = KFold(n_splits=int(np.rint(len(y)/size_test)),shuffle=True)
  gen = kf.split(X_copy)
  gens = []
  map_columns = X_copy.columns.to_list()
  if decider_type == 'history':
    histories = []
  elif decider_type == 'jump':
    feature_position_histories = []

  for generator in gen:
    gens.append(generator)
    if decider_type == 'history':
      histories.append([])
    
  generator_index = -1
  for generator in tqdm(gens):
    generator_index += 1
    if gridsearch and or_features:
      target_features,or_opportunities = automated_leave_k_out(X_copy.iloc[generator[0]], y.iloc[generator[0]], target_features = False,method = method,metric = metric,usage_map=False,intermediate_n = True,gridsearch=False,size_test = size_gridsearch_test,no_cv=no_cv,keep_going = keep_going,gridsearch_metric = gridsearch_metric,gridsearch_mode = True,or_features=True,decider_type=decider_type)
      local_X_copy = copy.deepcopy(X_copy)
      flatten_or_opportunities = set([i for abc in or_opportunities.values() for i in abc])
      print(flatten_or_opportunities)
      for opportunity in flatten_or_opportunities:
        iter_opportunity = iter(opportunity)
        opportunity_1 = next(iter_opportunity)
        opportunity_2 = next(iter_opportunity)
        local_X_copy[opportunity_1 + ' or ' + opportunity_2] = local_X_copy[opportunity_1] | local_X_copy[opportunity_2]
        if opportunity_1 + ' or ' + opportunity_2 not in map_columns:
          map_columns.append(opportunity_1 + ' or ' + opportunity_2)
      local_X_copy = pd.DataFrame(preprocessing.scale(local_X_copy),columns = local_X_copy.columns,index = local_X_copy.index)
      print(local_X_copy.columns)
    elif gridsearch and not or_features:
      target_features = automated_leave_k_out(X_copy.iloc[generator[0]], y.iloc[generator[0]], target_features = False,method = method,metric = metric,usage_map=False,intermediate_n = True,gridsearch=False,size_test = size_gridsearch_test,no_cv=no_cv,keep_going = keep_going,gridsearch_metric = gridsearch_metric,gridsearch_mode = True,or_features=False,decider_type=decider_type)
      local_X_copy = copy.deepcopy(pd.DataFrame(preprocessing.scale(X_copy),columns = X_copy.columns,index = X_copy.index))
    else:
      local_X_copy = copy.deepcopy(pd.DataFrame(preprocessing.scale(X_copy),columns = X_copy.columns,index = X_copy.index))
    #assert False #think about this a bit more
    if method == 'step_forward_selection':
      selection_counter = 0
      chosen = []
    elif method == 'forward_selection':
      chosen = []
    elif method == 'backward_selection':
      chosen = local_X_copy.columns.to_list()
    best_result = 0.0
    feature_positions = []
    while True:
      if method == 'step_forward_selection':
        selection_counter += 1
      if (method == 'forward_selection') or ((method =='step_forward_selection') and (selection_counter %3 != 0)):
        best_feature = ()
        results = []
        features_ = []
        for feature in [i for i in local_X_copy.columns if i not in chosen]:
          result,x_fit,test_right,model = find_next(chosen,feature,metric,generator,local_X_copy,y)
          results.append(result)
          features_.append(feature)
          if len(best_feature) == 0:
            best_feature = (feature,result,test_right,model)
          elif result > best_feature[1]:
            best_feature = (feature,result,test_right,model)
          if result > best_result:
            new_chosen = x_fit
            best_result = result
            best_test_right = test_right
            right_out = model
        if selection_counter %3 == 1:
          feature_positions.append([feature for _,feature in sorted(zip(results,features_),reverse=True)])
        if (target_features>0 and not keep_going) or (len(new_chosen) > len(chosen)) or (keep_going and (len(chosen) < len(local_X_copy.columns))): 
          if len(new_chosen) <= len(chosen): # first we try it the old way but if we keep_going, we still do the best we can.
            chosen = np.append(chosen,best_feature[0])
            best_test_right = best_feature[2]
            right_out = best_feature[3]
            new_chosen = chosen
          else:
            chosen = new_chosen
          if intermediate_n and ((not method == 'step_forward_selection') or len(chosen) == len(local_X_copy.columns)):
            if len(chosen) in test_rights_intermediate.keys():
              test_rights_intermediate[len(chosen)].append(best_test_right)
            else:
              test_rights_intermediate[len(chosen)] = [best_test_right]
            if len(chosen) in chosen_all_intermediate.keys():
              chosen_all_intermediate[len(chosen)].append(frozenset(chosen))
            else:
              chosen_all_intermediate[len(chosen)] = [frozenset(chosen)]
            if len(chosen) in out_intermediate.columns:
              out_intermediate.iloc[generator[1], out_intermediate.columns.get_loc(len(chosen))] = right_out
            else:
              out_intermediate[len(chosen)] = 0
              out_intermediate.iloc[generator[1], out_intermediate.columns.get_loc(len(chosen))] = right_out
          if (len(chosen) == target_features) and ((not method == 'step_forward_selection') or len(chosen) == len(local_X_copy.columns)):
            print(target_features)
            how_many.append(target_features)
            out.iloc[generator[1]] = right_out
            chosen_all.append(frozenset(chosen)) 
            test_rights.append(best_test_right)
            if not keep_going and (method == 'forward_selection'):
              break
        else:
          if not target_features:
            how_many.append(len(chosen))
            out.iloc[generator[1]] = right_out
            chosen_all.append(frozenset(chosen)) 
            test_rights.append(best_test_right)
          if (method != 'step_forward_selection') or (len(chosen) == len(local_X_copy.columns)):
            break
      if (method == 'backward_selection') or ((method =='step_forward_selection') and (selection_counter %3 == 0)):
        result,x_fit,test_right,model = remove_next(chosen,metric,generator,local_X_copy,y)
        new_chosen = x_fit
        best_result = result
        best_test_right = test_right
        right_out = model
        if not gridsearch_mode and or_features:
          print(new_chosen)
        if decider_type == 'history':
          histories[generator_index].append(new_chosen)
        if not len(new_chosen) == 0: 
          chosen = new_chosen
          if intermediate_n:
            if len(chosen) in test_rights_intermediate.keys():
              test_rights_intermediate[len(chosen)].append(best_test_right)
            else:
              test_rights_intermediate[len(chosen)] = [best_test_right]
            if len(chosen) in chosen_all_intermediate.keys():
              chosen_all_intermediate[len(chosen)].append(frozenset(chosen))
            else:
              chosen_all_intermediate[len(chosen)] = [frozenset(chosen)]
            if len(chosen) in out_intermediate.columns:
              out_intermediate.iloc[generator[1], out_intermediate.columns.get_loc(len(chosen))] = right_out
            else:
              out_intermediate[len(chosen)] = 0
              out_intermediate.iloc[generator[1], out_intermediate.columns.get_loc(len(chosen))] = right_out
          if len(chosen) == target_features:
            print(target_features)
            how_many.append(target_features)
            out.iloc[generator[1]] = right_out
            chosen_all.append(frozenset(chosen)) 
            test_rights.append(best_test_right)
            if not keep_going:
              break
          if len(chosen) == 0:
            break
          if ((method =='step_forward_selection') and (len(chosen) == len(local_X_copy.columns) - 1)):
            break
        else:
          if not target_features:
            how_many.append(len(chosen))
            out.iloc[generator[1]] = right_out
            chosen_all.append(frozenset(chosen)) 
            test_rights.append(best_test_right)
          break
    if or_features:
      feature_position_histories.append(feature_positions)
    if no_cv and not gridsearch_mode:
      break

  if gridsearch_mode:
    metrics = {}
    for i in np.arange(1,len(chosen_all_intermediate)+1,1):
      if np.sum(out_intermediate[i].isna()) > 0: # this is because if one of my Folds didn't go to that n_features, I don't want to look at it
        break # this should happen only once
      if gridsearch_metric == 'auc':
        metrics[i] = roc_auc_score(y,out_intermediate[i])
      elif gridsearch_metric == 'accuracy':
        metrics[i] = np.sum(test_rights_intermediate[i])/len(y)
      elif gridsearch_metric == 'binomial_fraying':
        fraying_set = set(chosen_all_intermediate[i])
        fraying_list = []
        for features in fraying_set:
          fraying_list.append(chosen_all_intermediate[i].count(features))
        fraying = np.sum([j**2/len(gens)**2 for j in fraying_list])
        if (i <= 6) or (i>=20):
          metrics[i] = 0.0
        else:
          metrics[i] = fraying
    n_best = max(metrics, key=metrics.get)
    if or_features:
      n_best_feature_set = set(chosen_all_intermediate[n_best])
      fraying_dicts = {}
      frayings = {}
      or_opportunities = {}
      and_opportunities = {}
      for i in np.arange(1,n_best,1):
        fraying_set = set(chosen_all_intermediate[i])
        fraying_dict = {}
        for features in fraying_set:
          fraying_dict[features] = chosen_all_intermediate[i].count(features)
        fraying = np.sum([j**2/len(gens)**2 for j in fraying_dict.values()])
        new_or_opportunities = []
        new_and_opportunities = []
        if decider_type == 'history':
          if len(frayings) > 0:
            if frayings[i-1] > fraying:
              abc = list(fraying_set)
              pair_gen = ((abc[x],abc[y]) for x in range(len(abc)) for y in range(len(abc)) if x<y)
              for pair in pair_gen:
                both = pair[0].intersection(pair[1])
                first_feature = next(iter(pair[0]-pair[1])) # next(iter()) just gets 'a' from frozenset({'a'})
                second_feature = next(iter(pair[1]-pair[0]))
                if both in fraying_dicts[i-1].keys() and first_feature not in not_categorical and second_feature not in not_categorical: # that means that len(xor(_,_)) = 2
                  print(first_feature,second_feature)
          #         # make sure they aren't put together in the end
          #         #n_best_targets = [n_best_features for n_best_features in set(chosen_all_intermediate[n_best]) if first_feature in n_best_features and second_feature in n_best_features]
          #         #decider = np.sum([chosen_all_intermediate[n_best].count(j) for j in n_best_targets]) >= fraying_dict[pair[0]] + fraying_dict[pair[1]] #here we don't think about the cases where a different branch develops both pair[0] and pair[1] later
          #         #decider = False # if we don't care if they come back together again afterwards because we didn't do it perfectly (see 311)
          #         #we could use history here to do decider:
                  decider = 0
                  for history in histories:
                    if first_feature in history[n_best-1] and second_feature in history[n_best-1] or first_feature not in history[n_best-1] and second_feature not in history[n_best-1]:
                      pass
                    else:
                      decider += 1
                  print(decider)
                  if decider >= decider_misses:
                    new_or_opportunities.append(frozenset([first_feature,second_feature]))
        #or we can see if a Feature position jumped more than decider_min_jumps backwards
        elif decider_type == 'jump':
          for feature_positions in feature_position_histories:
            for j in range(len(feature_positions)-1):
              if set(feature_positions[j+1]).issubset(set(feature_positions[j])): #otherwise two were taken and one was returned and we don't know what made the jump happen
                for feature_index in range(len(feature_positions[j+1])):
                  if feature_positions[j].index(feature_positions[j+1][feature_index]) + decider_min_jumps <= feature_index: #before position+decider_min_jumps <= now position #everything will jump one position because of the different length
                    print([feature_positions[j+1][feature_index],next(iter(set(feature_positions[j])-set(feature_positions[j+1])))])
                    new_or_opportunities.append(frozenset([feature_positions[j+1][feature_index],next(iter(set(feature_positions[j])-set(feature_positions[j+1])))]))
                  if feature_positions[j].index(feature_positions[j+1][feature_index]) - decider_min_jumps >= feature_index: #before position-decider_min_jumps >= now position #everything will jump one position because of the different length
                    new_and_opportunities.append(frozenset([feature_positions[j+1][feature_index],next(iter(set(feature_positions[j])-set(feature_positions[j+1])))]))         
        or_opportunities[i] = new_or_opportunities
        and_opportunities[i] = new_and_opportunities
        fraying_dicts[i] = fraying_dict
        frayings[i] = fraying
      print(or_opportunities)
      print(and_opportunities)
      return n_best,or_opportunities
    else:
      return n_best
  if no_cv:
    if intermediate_n:
      for i in np.arange(1,len(chosen_all_intermediate)+1,1):
        test_accuracy = np.sum(test_rights_intermediate[i])/len(y.iloc[gens[0][1]])
        try:
          auc = roc_auc_score(y.iloc[gens[0][1]],out_intermediate[i].iloc[gens[0][1]])
        except:
          auc = np.nan #here if test_size is too small and it's all 1 or all 0, the AUC won't work
        print(f'Length {i}')
        print(f'Accuracy = {test_accuracy}')
        print(f'AUC = {auc}')
    if usage_map:
      map = pd.Series(index = map_columns,dtype = 'float64')
      for feature in map_columns:
        map.loc[feature] = np.sum([1 for features in chosen_all if feature in features])/len(chosen_all)
      map.sort_values(ascending=False,inplace=True)    
      with pd.option_context('display.max_rows', None):
        print(map)
  else:
    print(decider_type)
    if intermediate_n:
      for i in np.arange(1,len(chosen_all_intermediate)+1,1):
        if np.sum(out_intermediate[i].isna()) > 0:
          break
        auc = roc_auc_score(y,out_intermediate[i])
        fraying_set = set(chosen_all_intermediate[i])
        fraying_list = []
        for features in fraying_set:
          fraying_list.append(chosen_all_intermediate[i].count(features))
        fraying = np.sum([i**2/len(gens)**2 for i in fraying_list])
        test_accuracy = np.sum(test_rights_intermediate[i])/len(y)
        print(f'Length {i}')
        if fraying == 1.0:
          print(fraying_set)
        print(f'Accuracy = {test_accuracy}')
        print(f'AUC = {auc}')
        print(f'"Fraying":{fraying}, {fraying_list}')
    aucs = []
    abc = pd.DataFrame(out,columns=['out'])
    abc['y'] = y
    for i in range(100):
      ddd = resample(abc)
      aucs.append(roc_auc_score(ddd['y'],ddd['out']))
    auc = np.mean(aucs)
    auc_error = np.std(aucs)
    test_accuracy = np.sum(test_rights)/len(y)
    fraying_set = set(chosen_all)
    fraying_list = []
    for features in fraying_set:
      fraying_list.append(chosen_all.count(features))
    fraying = np.sum([i**2/len(gens)**2 for i in fraying_list])
    if intermediate_n:
      print('Altogether:')
    if (np.sum(fraying_list) == i) and (fraying == 1.0):
      print(fraying_set)
    print(f'Accuracy = {test_accuracy}')
    print(f'AUC = {auc} +- {auc_error}')
    print(f'"Fraying":{fraying}, {fraying_list}')
    if usage_map:
      map = pd.Series(index = map_columns,dtype = 'float64')
      for feature in map_columns:
        map.loc[feature] = np.sum([1 for features in chosen_all if feature in features])/len(chosen_all)
      map.sort_values(ascending=False,inplace=True)    
      with pd.option_context('display.max_rows', None):
        print(map)
    plt.hist(how_many,bins=np.arange(np.min(how_many),np.max(how_many)+1,1)-0.5)
    plt.title('Chosen # Features for each Fold')
    plt.show()
    plt.close()
  return out

#we don't use these anymore
def hard_function(tau, X, y, orthogonal_to = [], show = False, debug = False, estimator = logistic_estimator_with_pytorch, pad = True):
  
  X_copy = copy.deepcopy(X)
  rows = X_copy.columns.to_list()
  ca_complete = all(x in rows for x in ca_list)
  thal_complete = all(x in rows for x in thal_list)
  cp_complete = all(x in rows for x in cp_list)
  slope_complete = all(x in rows for x in slope_list)
  restecg_complete = all(x in rows for x in restecg_list)

  if ca_complete == True:
    X_copy.drop(["ca_0.0"],axis = 1, inplace=True)
  if thal_complete == True:
    X_copy.drop(["thal_3.0"],axis = 1, inplace= True)
  if slope_complete == True:
    X_copy.drop(["slope_1.0"],axis = 1, inplace= True)
  if cp_complete == True:
    X_copy.drop(["cp_4.0"],axis = 1, inplace= True)
  if restecg_complete == True:
    X_copy.drop(["restecg_0.0"],axis = 1, inplace= True)
   
  X_copy['y'] = y

  x_fit = []
  for i in range(30):
    abc = resample(X_copy)
    abc_y = abc['y'].to_numpy()
    abc_x = abc.drop(['y'],axis=1)
    scaled_data = preprocessing.scale(abc_x)
    data = ExperimentData(scaled_data,abc_y)
    INPUT_SIZE = scaled_data.shape[1]
    EPOCHS=1 # Few epochs to avoid overfitting
    pred_train = {}
    HIDDEN_LAYER_SIZE = []
    data_loader = DataLoader(data, batch_size=scaled_data.shape[0])
    net = NNet(INPUT_SIZE, HIDDEN_LAYER_SIZE, sigmoid=True, loss = my_loss)
    net.optim = LBFGS(net.parameters(), history_size=10, max_iter=10,lr=2.0)
    net.train(data_loader, EPOCHS)
    x_fit.append(np.append(net.layers[0].weight[0].detach().numpy(),net.layers[0].bias[0].detach().numpy()))

  # result = cross_correlate(scaled_data,y,estimator=estimator,pad=pad)
  
  # train_mean = np.mean(result['train_accuracy'])
  # train_err_mean = np.std(result['train_accuracy'])
  # test_mean = np.mean(result['test_accuracy'])
  # test_err_mean = np.std(result['test_accuracy'])

  # if debug:
  #   print(train_mean) # test
  #   print(train_err_mean)

  #   print(test_mean) # train
  #   print(train_err_mean)

  X_copy.drop(['y'],axis=1,inplace=True)

  importance = np.mean(x_fit,axis=0)
  importance_stdev = np.std(x_fit,axis=0)[:-1]
  average = importance[-1]
  importance = importance[:-1]

  plt.rcParams['figure.figsize'] = [16,8]
  x_tick = X_copy.columns.to_list()

  if debug:

    fig = plt.figure()
    plt.bar(x_tick,importance,yerr=importance_stdev)
    plt.ylabel("Significance")
    plt.show()

  importance, importance_stdev,average,x_tick = complete_cats("ca","0.0",ca_complete,orthogonal_to,importance, importance_stdev,average,x_tick,df,X_copy,debug)
  importance, importance_stdev,average,x_tick = complete_cats("cp","4.0",cp_complete,orthogonal_to,importance, importance_stdev,average,x_tick,df,X_copy,debug)
  importance, importance_stdev,average,x_tick = complete_cats("thal","3.0",thal_complete,orthogonal_to,importance, importance_stdev,average,x_tick,df,X_copy,debug)
  importance, importance_stdev,average,x_tick = complete_cats("slope","1.0",slope_complete,orthogonal_to,importance, importance_stdev,average,x_tick,df,X_copy,debug)
  importance, importance_stdev,average,x_tick = complete_cats("restecg","0.0",restecg_complete,orthogonal_to,importance, importance_stdev,average,x_tick,df,X_copy,debug)

  std_count = np.array([importance[i]/importance_stdev[i] for i in range(len(importance))])
  x_tick = np.array(x_tick)
  sorted_std = sorted(std_count, key=abs,reverse=True)
  sorted_ticks = [tick for _,tick in sorted(zip(abs(std_count),x_tick),reverse=True)]
  if debug:
    plt.figure(figsize=(28,14))
    plt.bar(sorted_ticks,sorted_std)
    plt.ylabel("sorted std's")
    plt.show()
  out = np.array(sorted_ticks)[np.abs(sorted_std)>tau]
  if len(np.array(sorted_ticks)[np.abs(sorted_std)<tau]) == 0 or len(out) == 0:
    if show:
      plt.figure(figsize=(28,14))
      plt.bar(sorted_ticks,sorted_std)
      plt.ylabel("sorted std's")
      plt.show()
    return out;
  else: 
    if debug: print(out)
    return hard_function(tau, X[out], y, orthogonal_to=orthogonal_to, show = show, debug = debug)

#not used
def other_hard_function(X, y, orthogonal_to = [], show = False, debug = False, estimator = logistic_estimator_with_pytorch, pad = True,random_features = True, features_premade = []):
  
  X_copy = copy.deepcopy(X)
  # rows = X_copy.columns.to_list()
  # ca_complete = all(x in rows for x in ca_list)
  # thal_complete = all(x in rows for x in thal_list)
  # cp_complete = all(x in rows for x in cp_list)
  # slope_complete = all(x in rows for x in slope_list)
  # restecg_complete = all(x in rows for x in restecg_list)

  # if ca_complete == True:
  #   X_copy.drop(["ca_0.0"],axis = 1, inplace=True)
  # if thal_complete == True:
  #   X_copy.drop(["thal_3.0"],axis = 1, inplace= True)
  # if slope_complete == True:
  #   X_copy.drop(["slope_1.0"],axis = 1, inplace= True)
  # if cp_complete == True:
  #   X_copy.drop(["cp_4.0"],axis = 1, inplace= True)
  # if restecg_complete == True:
  #   X_copy.drop(["restecg_0.0"],axis = 1, inplace= True)

  X_copy['y'] = y

  hij = []
  did_appear = {}
  for feature in X.columns:
    did_appear[feature] = []

  for i in tqdm(range(2000)):
    random_features_list = []
    if random_features:
      random_features_list = [feature for feature in X.columns.drop(features_premade) if random.random() > 0.05]
    bcd = random_features_list+features_premade + ['y']
    abc = resample(X_copy[bcd])
    abc_y = abc['y'].to_numpy()
    abc_x = abc.drop(['y'],axis=1)
    scaled_data = preprocessing.scale(abc_x)
    
    model = LogisticRegression(max_iter = 1000,fit_intercept=True,solver='liblinear',C=1000)
    model.fit(abc_x,abc_y)
    x_fit = np.append(model.coef_[0],model.intercept_[0])

    # data = ExperimentData(scaled_data,abc_y)
    # INPUT_SIZE = scaled_data.shape[1]
    # EPOCHS=1 # Few epochs to avoid overfitting
    # pred_train = {}
    # HIDDEN_LAYER_SIZE = []
    # data_loader = DataLoader(data, batch_size=scaled_data.shape[0])
    # net = NNet(INPUT_SIZE, HIDDEN_LAYER_SIZE, sigmoid=True, loss = my_loss)
    # net.optim = LBFGS(net.parameters(), history_size=10, max_iter=10,lr=2.0)
    # net.train(data_loader, EPOCHS)
    # x_fit = np.append(net.layers[0].weight[0].detach().numpy(),net.layers[0].bias[0].detach().numpy())
    # hij.append(x_fit)
    features = random_features_list+features_premade
    for i in range(len(features)):
      did_appear[features[i]].append(x_fit[i])

  X_copy.drop(['y'],axis=1,inplace=True)

  importance = {}
  importance_stdev = {}
  for feature in X.columns:
    importance[feature] = np.mean(did_appear[feature])
    importance_stdev[feature] = np.std(did_appear[feature])

  plt.rcParams['figure.figsize'] = [16,8]
  x_tick = X_copy.columns.to_list()

  fig = plt.figure()
  plt.bar(x_tick,importance.values(),yerr=importance_stdev.values())
  plt.ylabel("Significance")
  plt.show()

  if debug:
    for feature in X.columns:
      plt.hist(did_appear[feature],label = feature)
      plt.legend()
      plt.show()
      plt.close()

  # importance, importance_stdev,average,x_tick = complete_cats("ca","0.0",ca_complete,orthogonal_to,importance, importance_stdev,average,x_tick,df,X_copy,debug)
  # importance, importance_stdev,average,x_tick = complete_cats("cp","4.0",cp_complete,orthogonal_to,importance, importance_stdev,average,x_tick,df,X_copy,debug)
  # importance, importance_stdev,average,x_tick = complete_cats("thal","3.0",thal_complete,orthogonal_to,importance, importance_stdev,average,x_tick,df,X_copy,debug)
  # importance, importance_stdev,average,x_tick = complete_cats("slope","1.0",slope_complete,orthogonal_to,importance, importance_stdev,average,x_tick,df,X_copy,debug)
  # importance, importance_stdev,average,x_tick = complete_cats("restecg","0.0",restecg_complete,orthogonal_to,importance, importance_stdev,average,x_tick,df,X_copy,debug)

  std_count = np.array([importance[i]/importance_stdev[i] for i in X.columns])
  x_tick = np.array(x_tick)
  sorted_std = sorted(std_count, key=abs,reverse=True)
  sorted_ticks = [tick for _,tick in sorted(zip(abs(std_count),x_tick),reverse=True)]

  plt.figure(figsize=(150,75))
  plt.bar(sorted_ticks,sorted_std)
  plt.ylabel("sorted std's")
  plt.show()
  
  return did_appear

## Data 'wrangling'

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data',header=None)

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [ ]:
columns = ["age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal","num"]       

* sex: sex (1 = male; 0 = female)



* cp: 

        -- Value 1: typical angina
        -- Value 2: atypical angina
        -- Value 3: non-anginal pain
        -- Value 4: asymptomatic

* trestbps: resting blood pressure (in mm Hg on admission to the hospital)

* chol: serum cholestoral in mg/dl

* fbs: (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)

* restecg: resting electrocardiographic results

        -- Value 0: normal
        -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST 
                    elevation or depression of > 0.05 mV)
        -- Value 2: showing probable or definite left ventricular hypertrophy
                    by Estes' criteria

* thalach: maximum heart rate achieved

* exang: exercise induced angina (1 = yes; 0 = no)

* oldpeak = ST depression induced by exercise relative to rest

* slope: the slope of the peak exercise ST segment
        -- Value 1: upsloping
        -- Value 2: flat
        -- Value 3: downsloping
* ca: number of major vessels (0-3) colored by flourosopy

* thal: 3 = normal; 6 = fixed defect; 7 = reversable defect

* num: diagnosis of heart disease (angiographic disease status)
        -- Value 0: < 50% diameter narrowing
        -- Value 1: > 50% diameter narrowing
        (in any major vessel: attributes 59 through 68 are vessels)


In [ ]:
df.columns = columns

In [ ]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [ ]:
df["ca"].value_counts()

0.0    176
1.0     65
2.0     38
3.0     20
?        4
Name: ca, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    float64
 1   sex       303 non-null    float64
 2   cp        303 non-null    float64
 3   trestbps  303 non-null    float64
 4   chol      303 non-null    float64
 5   fbs       303 non-null    float64
 6   restecg   303 non-null    float64
 7   thalach   303 non-null    float64
 8   exang     303 non-null    float64
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    float64
 11  ca        303 non-null    object 
 12  thal      303 non-null    object 
 13  num       303 non-null    int64  
dtypes: float64(11), int64(1), object(2)
memory usage: 33.3+ KB


Let's look at ca and thal.

In [ ]:
df["ca"].value_counts()

0.0    176
1.0     65
2.0     38
3.0     20
?        4
Name: ca, dtype: int64

The ? represent missing data

In [ ]:
df["thal"].value_counts()

3.0    166
7.0    117
6.0     18
?        2
Name: thal, dtype: int64

In [ ]:
df.loc[(df["ca"]=="?") | (df["thal"]=="?")]

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
87,53.0,0.0,3.0,128.0,216.0,0.0,2.0,115.0,0.0,0.0,1.0,0.0,?,0
166,52.0,1.0,3.0,138.0,223.0,0.0,0.0,169.0,0.0,0.0,1.0,?,3.0,0
192,43.0,1.0,4.0,132.0,247.0,1.0,2.0,143.0,1.0,0.1,2.0,?,7.0,1
266,52.0,1.0,4.0,128.0,204.0,1.0,0.0,156.0,1.0,1.0,2.0,0.0,?,2
287,58.0,1.0,2.0,125.0,220.0,0.0,0.0,144.0,0.0,0.4,2.0,?,7.0,0
302,38.0,1.0,3.0,138.0,175.0,0.0,0.0,173.0,0.0,0.0,1.0,?,3.0,0


In [ ]:
df.shape

(303, 14)

Imputing missing values (this is done once below). We get the same result using 'num' or not using 'num' so we can do this once and don't need to differentiate train and test.


In [ ]:
# #Because we have Accuracy = 85%, we only use data with more than 85% chance of being correct.
df.loc[87,'thal'] = 3.0 # chance of being correct is 99% - this becomes 98% when using num
# df.loc[166,'ca'] = 0.0 # chance is 75%
# df.loc[192,'ca'] = 0.0 # chance is 70#
# df.loc[266,'thal'] = 7.0 # chance of being correct is 71% - this becomes 88.5% when also using num but we can't always use num
# df.loc[287,'ca'] = 0.0 # chance is 69%
df.loc[302,'ca'] = 0.0 # chance is 95% - this becomes 94% when using num

In [ ]:
df.drop(index=166,inplace=True)
df.drop(index=192,inplace=True)
df.drop(index=266,inplace=True)
df.drop(index=287,inplace=True)

In [ ]:
df.loc[(df["ca"]=="?") | (df["thal"]=="?")]

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num


We don't have any more missing data and can start building the model. df_old is the original data frame.

In [ ]:
df['ca'] = df['ca'].astype(float, errors = 'raise')
df['thal'] = df['thal'].astype(float, errors = 'raise')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       299 non-null    float64
 1   sex       299 non-null    float64
 2   cp        299 non-null    float64
 3   trestbps  299 non-null    float64
 4   chol      299 non-null    float64
 5   fbs       299 non-null    float64
 6   restecg   299 non-null    float64
 7   thalach   299 non-null    float64
 8   exang     299 non-null    float64
 9   oldpeak   299 non-null    float64
 10  slope     299 non-null    float64
 11  ca        299 non-null    float64
 12  thal      299 non-null    float64
 13  num       299 non-null    int64  
dtypes: float64(13), int64(1)
memory usage: 35.0 KB


### **Split data into dependent and independent variables.**

In [ ]:
y = df['num'].copy()
X = df.drop('num',axis=1).copy()
display(X)
display(y)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0


0      0
1      2
2      1
3      0
4      0
      ..
298    1
299    2
300    3
301    1
302    0
Name: num, Length: 299, dtype: int64

In [ ]:
y.value_counts()

0    162
1     54
2     35
3     35
4     13
Name: num, dtype: int64

Why is the outcome between 0 and 4? It should just be 0 or 1. Even the website doesn't know.

Let's look at the data:

### *one-hot encoding*

In [ ]:
for i in X.columns:
  display(X[i].value_counts())

58.0    18
57.0    17
54.0    16
59.0    14
60.0    12
51.0    12
56.0    11
62.0    11
44.0    11
52.0    11
64.0    10
41.0    10
67.0     9
63.0     9
42.0     8
45.0     8
53.0     8
55.0     8
61.0     8
65.0     8
50.0     7
66.0     7
43.0     7
48.0     7
46.0     7
47.0     5
49.0     5
70.0     4
68.0     4
35.0     4
39.0     4
69.0     3
71.0     3
40.0     3
34.0     2
37.0     2
38.0     2
29.0     1
77.0     1
74.0     1
76.0     1
Name: age, dtype: int64

1.0    202
0.0     97
Name: sex, dtype: int64

4.0    142
3.0     85
2.0     49
1.0     23
Name: cp, dtype: int64

120.0    37
130.0    36
140.0    32
110.0    19
150.0    17
138.0    11
160.0    11
128.0    11
125.0    10
112.0     9
132.0     7
118.0     7
124.0     6
108.0     6
135.0     6
152.0     5
134.0     5
145.0     5
100.0     4
170.0     4
122.0     4
126.0     3
136.0     3
115.0     3
180.0     3
142.0     3
105.0     3
102.0     2
146.0     2
144.0     2
148.0     2
178.0     2
94.0      2
165.0     1
123.0     1
114.0     1
154.0     1
156.0     1
106.0     1
155.0     1
172.0     1
200.0     1
101.0     1
129.0     1
192.0     1
158.0     1
104.0     1
174.0     1
117.0     1
164.0     1
Name: trestbps, dtype: int64

197.0    6
234.0    6
269.0    5
204.0    5
212.0    5
        ..
247.0    1
340.0    1
160.0    1
394.0    1
131.0    1
Name: chol, Length: 152, dtype: int64

...

0.0    202
1.0     97
Name: exang, dtype: int64

0.0    98
1.2    17
0.6    14
1.4    13
0.8    13
1.0    13
0.2    12
1.6    11
1.8    10
2.0     9
0.4     8
0.1     6
2.8     6
2.6     6
1.9     5
0.5     5
3.0     5
1.5     5
3.6     4
2.2     4
3.4     3
0.9     3
2.4     3
0.3     3
4.0     3
1.1     2
4.2     2
2.3     2
2.5     2
3.2     2
5.6     1
2.9     1
6.2     1
2.1     1
1.3     1
3.1     1
3.8     1
0.7     1
3.5     1
4.4     1
Name: oldpeak, dtype: int64

1.0    141
2.0    137
3.0     21
Name: slope, dtype: int64

0.0    176
1.0     65
2.0     38
3.0     20
Name: ca, dtype: int64

3.0    166
7.0    115
6.0     18
Name: thal, dtype: int64

- [ ] age is a float
- [ ] sex is a category (0,1)
- [ ] cp is a category (1,2,3,4)
- [ ] trestbps is a float
- [ ] chol is a float
- [ ] fbs is a category (0,1)
- [ ] restecg is a category (0,1,2) !! 1 only appears 4 times
- [ ] thalach is a float
- [ ] exang is a category (0,1)
- [ ] oldpeak is a float
- [ ] slope is a category (1,2,3)
- [ ] ca is a category (0,1,2,3)
- [ ] thal is a category (3,6,7)


I need to one-hot encode:

cp,restecg,slope,ca,thal

my other categories are:

sex,fbs,exang

my other floats are:

age, trestbps,chol,thalach,oldpeak


Let's do cp:

In [ ]:
X["cp"].value_counts()

4.0    142
3.0     85
2.0     49
1.0     23
Name: cp, dtype: int64

We need to set cp1 to 1 if cp = 1 or 0 if it's not:

In [ ]:
X_old = X.copy()
for row in X.index:
  X.loc[row,"cp1"] = X.loc[row,"cp"] == 1.0
  X.loc[row,"cp2"] = X.loc[row,"cp"] == 2.0
  X.loc[row,"cp3"] = X.loc[row,"cp"] == 3.0
  X.loc[row,"cp4"] = X.loc[row,"cp"] == 4.0
X.drop("cp",axis=1,inplace=True)
X

,age,sex,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,cp1,cp2,cp3,cp4
0,63.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,True,False,False,False
1,67.0,1.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,False,False,False,True
2,67.0,1.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,False,False,False,True
3,37.0,1.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,False,False,True,False
4,41.0,0.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,True,False,False,False
299,68.0,1.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,False,False,False,True
300,57.0,1.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,False,False,False,True
301,57.0,0.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,False,True,False,False


That looks good, let's check if any are all False (that means, the cp was something other than what we looked for above).

In [ ]:
maskcp = ((X["cp1"] == False) & (X["cp2"] == False) & (X["cp3"] == False) & (X["cp4"] == False))
X.loc[maskcp]

,age,sex,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,cp1,cp2,cp3,cp4


That means we got them all. Finally, let's make sure that each new column has some true values:

In [ ]:
display(X["cp1"].value_counts())
display(X["cp2"].value_counts())
display(X["cp3"].value_counts())
display(X["cp4"].value_counts())

False    276
True      23
Name: cp1, dtype: int64

False    250
True      49
Name: cp2, dtype: int64

False    214
True      85
Name: cp3, dtype: int64

False    157
True     142
Name: cp4, dtype: int64

Now we're done with cp, let's do the other ones.


In [ ]:
for row in X.index:
  X.loc[row,"restecg0"] = X.loc[row,"restecg"] == 0.0
  X.loc[row,"restecg1"] = X.loc[row,"restecg"] == 1.0
  X.loc[row,"restecg2"] = X.loc[row,"restecg"] == 2.0
X.drop("restecg",axis=1,inplace=True)

for row in X.index:
  X.loc[row,"slope1"] = X.loc[row,"slope"] == 1.0
  X.loc[row,"slope2"] = X.loc[row,"slope"] == 2.0
  X.loc[row,"slope3"] = X.loc[row,"slope"] == 3.0
X.drop("slope",axis=1,inplace=True)

for row in X.index:
  X.loc[row,"ca0"] = X.loc[row,"ca"] == 0.0
  X.loc[row,"ca1"] = X.loc[row,"ca"] == 1.0
  X.loc[row,"ca2"] = X.loc[row,"ca"] == 2.0
  X.loc[row,"ca3"] = X.loc[row,"ca"] == 3.0
X.drop("ca",axis=1,inplace=True)

for row in X.index:
  X.loc[row,"thal3"] = X.loc[row,"thal"] == 3.0
  X.loc[row,"thal6"] = X.loc[row,"thal"] == 6.0
  X.loc[row,"thal7"] = X.loc[row,"thal"] == 7.0
X.drop("thal",axis=1,inplace=True)
X

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,cp1,cp2,...,slope1,slope2,slope3,ca0,ca1,ca2,ca3,thal3,thal6,thal7
0,63.0,1.0,145.0,233.0,1.0,150.0,0.0,2.3,True,False,...,False,False,True,True,False,False,False,False,True,False
1,67.0,1.0,160.0,286.0,0.0,108.0,1.0,1.5,False,False,...,False,True,False,False,False,False,True,True,False,False
2,67.0,1.0,120.0,229.0,0.0,129.0,1.0,2.6,False,False,...,False,True,False,False,False,True,False,False,False,True
3,37.0,1.0,130.0,250.0,0.0,187.0,0.0,3.5,False,False,...,False,False,True,True,False,False,False,True,False,False
4,41.0,0.0,130.0,204.0,0.0,172.0,0.0,1.4,False,True,...,True,False,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,110.0,264.0,0.0,132.0,0.0,1.2,True,False,...,False,True,False,True,False,False,False,False,False,True
299,68.0,1.0,144.0,193.0,1.0,141.0,0.0,3.4,False,False,...,False,True,False,False,False,True,False,False,False,True
300,57.0,1.0,130.0,131.0,0.0,115.0,1.0,1.2,False,False,...,False,True,False,False,True,False,False,False,False,True
301,57.0,0.0,130.0,236.0,0.0,174.0,0.0,0.0,False,True,...,False,True,False,False,True,False,False,True,False,False


Let's check again that we got everything:

In [ ]:
maskrestecg = ((X["restecg0"] == False) & (X["restecg1"] == False) & (X["restecg2"] == False))
display(X.loc[maskrestecg])
maskslope = ((X["slope1"] == False) & (X["slope2"] == False) & (X["slope3"] == False))
display(X.loc[maskslope])
maskca = ((X["ca0"] == False) & (X["ca1"] == False) & (X["ca2"] == False) & (X["ca3"] == False))
display(X.loc[maskca])
maskthal = ((X["thal3"] == False) & (X["thal6"] == False) & (X["thal7"] == False))
display(X.loc[maskthal])

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,cp1,cp2,...,slope1,slope2,slope3,ca0,ca1,ca2,ca3,thal3,thal6,thal7


,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,cp1,cp2,...,slope1,slope2,slope3,ca0,ca1,ca2,ca3,thal3,thal6,thal7


,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,cp1,cp2,...,slope1,slope2,slope3,ca0,ca1,ca2,ca3,thal3,thal6,thal7


,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,cp1,cp2,...,slope1,slope2,slope3,ca0,ca1,ca2,ca3,thal3,thal6,thal7


Looks like we didn't do ca and thal correctly.

In [ ]:
X_old['ca']==0.0

0       True
1      False
2      False
3       True
4       True
       ...  
298     True
299    False
300    False
301    False
302     True
Name: ca, Length: 299, dtype: bool

The problem is that ca is still an object.

In [ ]:
X = X_old.astype({"ca": "float64", "thal": "float64"}).copy()

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 302
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       299 non-null    float64
 1   sex       299 non-null    float64
 2   cp        299 non-null    float64
 3   trestbps  299 non-null    float64
 4   chol      299 non-null    float64
 5   fbs       299 non-null    float64
 6   restecg   299 non-null    float64
 7   thalach   299 non-null    float64
 8   exang     299 non-null    float64
 9   oldpeak   299 non-null    float64
 10  slope     299 non-null    float64
 11  ca        299 non-null    float64
 12  thal      299 non-null    float64
dtypes: float64(13)
memory usage: 40.8 KB


In [ ]:
X['ca']==0.0

0       True
1      False
2      False
3       True
4       True
       ...  
298     True
299    False
300    False
301    False
302     True
Name: ca, Length: 299, dtype: bool

Now we do it all again:


In [ ]:
for row in X.index:
  X.loc[row,"cp1"] = X.loc[row,"cp"] == 1.0
  X.loc[row,"cp2"] = X.loc[row,"cp"] == 2.0
  X.loc[row,"cp3"] = X.loc[row,"cp"] == 3.0
  X.loc[row,"cp4"] = X.loc[row,"cp"] == 4.0
X.drop("cp",axis=1,inplace=True)

for row in X.index:
  X.loc[row,"restecg0"] = X.loc[row,"restecg"] == 0.0
  X.loc[row,"restecg1"] = X.loc[row,"restecg"] == 1.0
  X.loc[row,"restecg2"] = X.loc[row,"restecg"] == 2.0
X.drop("restecg",axis=1,inplace=True)

for row in X.index:
  X.loc[row,"slope1"] = X.loc[row,"slope"] == 1.0
  X.loc[row,"slope2"] = X.loc[row,"slope"] == 2.0
  X.loc[row,"slope3"] = X.loc[row,"slope"] == 3.0
X.drop("slope",axis=1,inplace=True)

for row in X.index:
  X.loc[row,"ca0"] = X.loc[row,"ca"] == 0.0
  X.loc[row,"ca1"] = X.loc[row,"ca"] == 1.0
  X.loc[row,"ca2"] = X.loc[row,"ca"] == 2.0
  X.loc[row,"ca3"] = X.loc[row,"ca"] == 3.0
X.drop("ca",axis=1,inplace=True)

for row in X.index:
  X.loc[row,"thal3"] = X.loc[row,"thal"] == 3.0
  X.loc[row,"thal6"] = X.loc[row,"thal"] == 6.0
  X.loc[row,"thal7"] = X.loc[row,"thal"] == 7.0
X.drop("thal",axis=1,inplace=True)

maskcp = ((X["cp1"] == False) & (X["cp2"] == False) & (X["cp3"] == False) & (X["cp4"] == False))
X.loc[maskcp]
maskrestecg = ((X["restecg0"] == False) & (X["restecg1"] == False) & (X["restecg2"] == False))
display(X.loc[maskrestecg])
maskslope = ((X["slope1"] == False) & (X["slope2"] == False) & (X["slope3"] == False))
display(X.loc[maskslope])
maskca = ((X["ca0"] == False) & (X["ca1"] == False) & (X["ca2"] == False) & (X["ca3"] == False))
display(X.loc[maskca])
maskthal = ((X["thal3"] == False) & (X["thal6"] == False) & (X["thal7"] == False))
display(X.loc[maskthal])

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,cp1,cp2,...,slope1,slope2,slope3,ca0,ca1,ca2,ca3,thal3,thal6,thal7


,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,cp1,cp2,...,slope1,slope2,slope3,ca0,ca1,ca2,ca3,thal3,thal6,thal7


,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,cp1,cp2,...,slope1,slope2,slope3,ca0,ca1,ca2,ca3,thal3,thal6,thal7


,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,cp1,cp2,...,slope1,slope2,slope3,ca0,ca1,ca2,ca3,thal3,thal6,thal7


Great, now for checking the second part:

In [ ]:
display(X["cp1"].value_counts())
display(X["cp2"].value_counts())
display(X["cp3"].value_counts())
display(X["cp4"].value_counts())
display(X["restecg0"].value_counts())
display(X["restecg1"].value_counts())
display(X["restecg2"].value_counts())
display(X["slope1"].value_counts())
display(X["slope2"].value_counts())
display(X["slope3"].value_counts())
display(X["ca0"].value_counts())
display(X["ca1"].value_counts())
display(X["ca2"].value_counts())
display(X["ca3"].value_counts())
display(X["thal3"].value_counts())
display(X["thal6"].value_counts())
display(X["thal7"].value_counts())

False    276
True      23
Name: cp1, dtype: int64

False    250
True      49
Name: cp2, dtype: int64

False    214
True      85
Name: cp3, dtype: int64

False    157
True     142
Name: cp4, dtype: int64

False    151
True     148
Name: restecg0, dtype: int64

...

False    261
True      38
Name: ca2, dtype: int64

False    279
True      20
Name: ca3, dtype: int64

True     166
False    133
Name: thal3, dtype: int64

False    281
True      18
Name: thal6, dtype: int64

False    184
True     115
Name: thal7, dtype: int64

Noticable is that restecg0 only has four True values.

### Or we could just use get_dummies:



In [ ]:
X_encoded = pd.get_dummies(X_old,columns = ["cp","restecg","slope","ca","thal"])
X_encoded["exang"] = X_encoded["exang"].astype(int)
X_encoded["fbs"] = X_encoded["fbs"].astype(int)
X_encoded["sex"] = X_encoded["sex"].astype(int)
X_encoded

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,cp_1.0,cp_2.0,...,slope_1.0,slope_2.0,slope_3.0,ca_0.0,ca_1.0,ca_2.0,ca_3.0,thal_3.0,thal_6.0,thal_7.0
0,63.0,1,145.0,233.0,1,150.0,0,2.3,1,0,...,0,0,1,1,0,0,0,0,1,0
1,67.0,1,160.0,286.0,0,108.0,1,1.5,0,0,...,0,1,0,0,0,0,1,1,0,0
2,67.0,1,120.0,229.0,0,129.0,1,2.6,0,0,...,0,1,0,0,0,1,0,0,0,1
3,37.0,1,130.0,250.0,0,187.0,0,3.5,0,0,...,0,0,1,1,0,0,0,1,0,0
4,41.0,0,130.0,204.0,0,172.0,0,1.4,0,1,...,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1,110.0,264.0,0,132.0,0,1.2,1,0,...,0,1,0,1,0,0,0,0,0,1
299,68.0,1,144.0,193.0,1,141.0,0,3.4,0,0,...,0,1,0,0,0,1,0,0,0,1
300,57.0,1,130.0,131.0,0,115.0,1,1.2,0,0,...,0,1,0,0,1,0,0,0,0,1
301,57.0,0,130.0,236.0,0,174.0,0,0.0,0,1,...,0,1,0,0,1,0,0,1,0,0


### Let's now deal with y:

In [ ]:
y_mask = y > 0
y[y_mask] = 1
y

0      0
1      1
2      1
3      0
4      0
      ..
298    1
299    1
300    1
301    1
302    0
Name: num, Length: 299, dtype: int64

That way, we only have 1 and 0, not 2,3,4.